In [ ]:
import pandas as pd
import os
import re
import ast
import string
#!pip install nltk
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from tqdm import tqdm
#!pip install contractions
from contractions import contractions_dict  
#!pip install emoji
import emoji
#!pip install vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from collections import defaultdict
import numpy as np

# Emolex total words are inaccurate bc some words have 0, proportions above 1 are also inaccurate
# pos neg and neu are also inaccurate in final_df here, but i fixed it in sentiment (check before rerunning, and check after)
# some words can have multiple emotions
# add keyword flag


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\yeahs\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


     ---------------------------------------- 0.0/41.5 kB ? eta -:--:--
     ---------------------------- --------- 30.7/41.5 kB 660.6 kB/s eta 0:00:01
     -------------------------------------- 41.5/41.5 kB 505.0 kB/s eta 0:00:00
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---- ----------------------------------- 0.2/1.5 MB 3.1 MB/s eta 0:00:01
   --------- ------------------------------ 0.4/1.5 MB 4.6 MB/s eta 0:00:01
   ---------------- ----------------------- 0.6/1.5 MB 5.7 MB/s eta 0:00:01
   ---------------------- ----------------- 0.9/1.5 MB 4.9 MB/s eta 0:00:01
   ------------------------------- -------- 1.2/1.5 MB 5.0 MB/s eta 0:00:01
   ---------------------------------------- 1.5/1.5 MB 6.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/274.0 kB ? eta -:--:--
   ---------------------------------------- 274.0/274.0 kB 8.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/97.9 kB ? eta -:--:--
   -------------------

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\yeahs\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\yeahs\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\yeahs\AppData\Roaming\nltk_data...
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\yeahs\AppData\Roaming\nltk_data...


   ---------------------------------------- 0.0/289.9 kB ? eta -:--:--
   ----- ---------------------------------- 41.0/289.9 kB 1.9 MB/s eta 0:00:01
   ----------------------------------- ---- 256.0/289.9 kB 3.9 MB/s eta 0:00:01
   ---------------------------------------- 289.9/289.9 kB 3.6 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\yeahs\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


   ---------------------------------------- 0.0/431.4 kB ? eta -:--:--
    --------------------------------------- 10.2/431.4 kB ? eta -:--:--
   ----- ---------------------------------- 61.4/431.4 kB 1.1 MB/s eta 0:00:01
   ------------------------- -------------- 276.5/431.4 kB 2.8 MB/s eta 0:00:01
   ---------------------------------------- 431.4/431.4 kB 3.0 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\yeahs\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


   ---------------------------------------- 0.0/126.0 kB ? eta -:--:--
   ------ -------------------------------- 20.5/126.0 kB 640.0 kB/s eta 0:00:01
   --------------- ----------------------- 51.2/126.0 kB 660.6 kB/s eta 0:00:01
   ---------------------------------------- 126.0/126.0 kB 1.2 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\yeahs\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


## Load and clean the data - as of 7/22

In [13]:
# Function to clean the content based on the keywords
def clean_content(content):
    # Remove content after specific keywords including the keywords
    keywords = ["Comments are closed", "Choked by their cap", "Print"]
    for keyword in keywords:
        last_occurrence = content.rfind(keyword)
        if last_occurrence != -1:
            content = content[:last_occurrence]

    # Remove " / Daily Nexus" and the two words before it
    pattern_daily_nexus = r'\b\w+\b \b\w+\b / Daily Nexus'
    content = re.sub(pattern_daily_nexus, '', content)

    # Remove the phrase "Nexus File Photo"
    content = content.replace("Nexus File Photo", "")

    # Clean special character 
    content = content.replace('', "")

    # Remove "Escrito por:" and the two words following it
    pattern_escrito_por = r'Escrito por: \b\w+\b \b\w+\b'
    content = re.sub(pattern_escrito_por, '', content)

    return content.strip()

# Function to read CSV with different encodings
def read_csv_with_encoding(file_path):
    encodings = ['utf-8', 'latin1', 'iso-8859-1']
    for encoding in encodings:
        try:
            return pd.read_csv(file_path, encoding=encoding)
        except UnicodeDecodeError:
            continue
    raise UnicodeDecodeError(f"Unable to decode the file: {file_path}")

# Combine sections and keep data from the first row among duplicates
def combine_rows(group):
    combined_section = list(group['Section'].unique())
    first_row = group.iloc[0].copy()
    first_row['Section'] = combined_section
    return first_row

# Function to clean the 'Authors' column
def clean_authors(authors_str):
    try:
        # Convert the string representation of the list into an actual list
        authors_list = ast.literal_eval(authors_str)
        # Ensure it's a list and remove single quotes if present
        if isinstance(authors_list, list):
            cleaned_list = [author.replace("'", "").strip() for author in authors_list]
            return cleaned_list
    except (ValueError, SyntaxError):
        pass
    return authors_str  # Return the original string if conversion fails

In [70]:
# Path to the directory containing the CSV files
dir_path = 'scraped_files'

# List all files in the directory
files = [f for f in os.listdir(dir_path) if f.endswith('.csv')]

# Read and concatenate all CSV files
df_list = []
for file in files:
    file_path = os.path.join(dir_path, file)
    df = read_csv_with_encoding(file_path)
    df_list.append(df)

# Concatenate all dataframes
combined_df = pd.concat(df_list, ignore_index=True)

# Keep Only 0 - 6 columns
combined_df = combined_df.iloc[:, 0:7]

# Remove dirty values
combined_df = combined_df[combined_df['Section'].map(combined_df['Section'].value_counts()) > 1]

# Convert Date and Time to datetime
combined_df['Date'] = pd.to_datetime(combined_df['Date'])
combined_df['Time'] = pd.to_datetime(combined_df['Time']).dt.time

# Define the mapping from old section names to formal names
section_mapping = {
    'news': 'News',
    'sports': 'Sports',
    'artsweek': 'Artsweek',
    'daily-stench': 'Daily Stench',
    'opinion': 'Opinion',
    'on-the-menu': 'On The Menu',
    'science': 'Science & Tech',
    'photo': 'Photo',
    'video': 'Video',
    'data': 'Data',
    'games': 'Games',
    'la-vista': 'La Vista'
}

# Replace the section names with the formal names
combined_df['Section'] = combined_df['Section'].replace(section_mapping)


# In content, remove just the words after the last "Print" in the content, "Comments are closed", or "Choked by their cap"
combined_df['Content'] = combined_df['Content'].apply(clean_content)

# Apply the cleaning function to the 'Authors' column
combined_df['Authors'] = combined_df['Authors'].apply(clean_authors)

# Group by URL and apply the combination function
combined_df = combined_df.groupby('URL', as_index=False).apply(combine_rows).reset_index(drop=True)

# Drop original duplicates from the DataFrame
combined_df = combined_df.drop_duplicates('URL', keep='first')

# Append the combined rows back to the DataFrame
combined_df = pd.concat([combined_df, combined]).drop_duplicates('URL', keep='first').reset_index(drop=True)

# Save the combined dataframe to a new CSV file if needed
combined_df.to_csv('data_storage/combined_df.csv', index=False)

# Show all columns in pandas
pd.set_option('display.max_columns', None)

combined_df

,Section,Title,Date,Time,Authors,URL,Content
0,[Sports],UCSB End Non-Conf. Portion of Season in Winnin...,2018-01-01,00:37:00,[Spencer Ault],https://dailynexus.com/2018-01-01/80339/,"After breaking through the defense, Jalen Cant..."
1,[Sports],Gauchos to Host Long Beach and Cal Poly to Ope...,2018-01-02,00:47:00,[Calvin Thrift],https://dailynexus.com/2018-01-02/gauchos-to-h...,"Sarah Porter dribbles the ball, looking for a ..."
2,[Sports],Gauchos Outlast 49ers in Conference Opener,2018-01-05,19:30:00,[Calvin Thrift],https://dailynexus.com/2018-01-05/gauchos-outl...,Sarah Porter brings the ball up the court. Des...
3,[News],Earthquake Measuring 3.8 Reported Near Isla Vi...,2018-01-06,19:49:00,[Hannah Jackson],https://dailynexus.com/2018-01-06/earthquake-m...,A 3.8 magnitude earthquake was reported off th...
4,[Artsweek],The Friday Review: Saturation III by Brockhampton,2018-01-06,15:14:00,[Konrad Neithercutt],https://dailynexus.com/2018-01-06/the-friday-r...,The Internet's favorite boy band just solidifi...
...,...,...,...,...,...,...,...
6560,[Daily Stench],Help! The girls on the high school tours scare me,2024-07-14,04:20:00,[Joseph R. Biden],https://dailynexus.com/2024-07-14/help-the-gir...,It was a classic spring quarter Sunday: I woke...
6561,[Artsweek],Cage the Elephant at the Santa Barbara Bowl: A...,2024-07-16,11:53:00,[Lauren Chiou],https://dailynexus.com/2024-07-16/cage-the-ele...,American rock band Cage the Elephant performed...
6562,[Artsweek],"""Mustard Man"": The ""post coming-of-age-story""...",2024-07-16,16:05:00,[Kyra Schimpf],https://dailynexus.com/2024-07-16/mustard-man-...,"Courtesy of ""Mustard Man"" crew Fresh off its C..."
6563,[On The Menu],Strawberry Avocado Toast: A Summer Delight,2024-07-18,10:00:00,[Pihu Jain],https://dailynexus.com/2024-07-18/117146/,"A perfect blend of the sweet and savory, this ..."


## Processing

In [152]:
# Function to expand contractions
def expand_contractions(text, contractions_dict=contractions_dict):
    contractions_pattern = re.compile('({})'.format('|'.join(contractions_dict.keys())), 
                                      flags=re.IGNORECASE|re.DOTALL)
    def expand_match(contraction):
        match = contraction.group(0)
        expanded_contraction = contractions_dict.get(match.lower())
        return expanded_contraction

    expanded_text = contractions_pattern.sub(expand_match, text)
    return expanded_text

# Function to remove URLs, mentions, hashtags, and special characters
def clean_text(text):
    text = re.sub(r"http\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)  # Remove URLs
    text = re.sub(r'\@\w+|\#','', text)  # Remove mentions and hashtags
    text = re.sub(r'[^A-Za-z0-9\sáéíóúñÁÉÍÓÚÑ]', '', text)  # Remove special characters except accented characters
    text = emoji.demojize(text)  # Convert emojis to text
    return text

# Function to preprocess the text in English
def preprocess_text(text):
    if isinstance(text, str):
        # Replace hyphens with spaces
        text = text.replace('-', ' ')
        # Clean text
        text = clean_text(text)
        # Expand contractions
        text = expand_contractions(text)
        # Convert to lowercase
        text = text.lower()
        # Remove punctuation
        text = text.translate(str.maketrans('', '', string.punctuation))
        # Tokenize the text
        tokens = word_tokenize(text)
        # Remove stopwords and numbers
        tokens = [word for word in tokens if word.isalpha() and word not in stopwords.words('english')]
        # Lemmatization
        lemmatizer = WordNetLemmatizer()
        tokens = [lemmatizer.lemmatize(word) for word in tokens]
        # Remove extra whitespace
        text = ' '.join(tokens).strip()
        return text
    else:
        return ''

# Function to preprocess the text in Spanish
def preprocess_text_spanish(text):
    if isinstance(text, str):
        # Replace hyphens with spaces
        text = text.replace('-', ' ')
        # Clean text
        text = clean_text(text)
        # Convert to lowercase
        text = text.lower()
        # Remove punctuation
        text = text.translate(str.maketrans('', '', string.punctuation))
        # Tokenize the text
        tokens = word_tokenize(text)
        # Remove stopwords and numbers
        spanish_stopwords = stopwords.words('spanish')
        tokens = [word for word in tokens if word.isalpha() and word not in spanish_stopwords]
        # Lemmatization is optional and complex for Spanish without proper libraries
        text = ' '.join(tokens).strip()
        return text
    else:
        return ''

In [153]:
# Load data
nlp_df = pd.read_csv('data_storage/combined_df.csv')
nlp_df['Authors'] = nlp_df['Authors'].apply(clean_authors)
nlp_df['Section'] = nlp_df['Section'].apply(clean_authors)

# Check if title and content is Spanish
nlp_df['is_spanish'] = nlp_df['Section'].apply(pd.Series).stack().str.contains('La Vista', case=False).groupby(level=0).any()

# Applying the preprocess_text function to the 'Content' column
tqdm.pandas()

# Preprocess Title with progress bar
nlp_df['Processed_Title'] = nlp_df.progress_apply(lambda x: preprocess_text_spanish(x['Title']) if x['is_spanish'] else preprocess_text(x['Title']), axis=1)

# Preprocess Content with progress bar
nlp_df['Processed_Content'] = nlp_df.progress_apply(lambda x: preprocess_text_spanish(x['Content']) if x['is_spanish'] else preprocess_text(x['Content']), axis=1)

# Save the preprocessed DataFrame to a new CSV file if needed
nlp_df.to_csv('data_storage/nlp_df.csv', index=False)

nlp_df

100%|██████████| 6565/6565 [11:34<00:00,  9.45it/s]


,Section,Title,Date,Time,Authors,URL,Content,is_spanish,Processed_Title,Processed_Content
0,[Sports],UCSB End Non-Conf. Portion of Season in Winnin...,2018-01-01,00:37:00,[Spencer Ault],https://dailynexus.com/2018-01-01/80339/,"After breaking through the defense, Jalen Cant...",False,ucsb end non conf portion season winning fashion,breaking defense jalen canty prepares shoot du...
1,[Sports],Gauchos to Host Long Beach and Cal Poly to Ope...,2018-01-02,00:47:00,[Calvin Thrift],https://dailynexus.com/2018-01-02/gauchos-to-h...,"Sarah Porter dribbles the ball, looking for a ...",False,gaucho host long beach cal poly open big west ...,sarah porter dribble ball looking pas non conf...
2,[Sports],Gauchos Outlast 49ers in Conference Opener,2018-01-05,19:30:00,[Calvin Thrift],https://dailynexus.com/2018-01-05/gauchos-outl...,Sarah Porter brings the ball up the court. Des...,False,gaucho outlast conference opener,sarah porter brings ball court despite another...
3,[News],Earthquake Measuring 3.8 Reported Near Isla Vi...,2018-01-06,19:49:00,[Hannah Jackson],https://dailynexus.com/2018-01-06/earthquake-m...,A 3.8 magnitude earthquake was reported off th...,False,earthquake measuring reported near isla vista ...,magnitude earthquake reported coast isla vista...
4,[Artsweek],The Friday Review: Saturation III by Brockhampton,2018-01-06,15:14:00,[Konrad Neithercutt],https://dailynexus.com/2018-01-06/the-friday-r...,The Internet's favorite boy band just solidifi...,False,friday review saturation iii brockhampton,internet favorite boy band solidified hip hop ...
...,...,...,...,...,...,...,...,...,...,...
6560,[Daily Stench],Help! The girls on the high school tours scare me,2024-07-14,04:20:00,[Joseph R. Biden],https://dailynexus.com/2024-07-14/help-the-gir...,It was a classic spring quarter Sunday: I woke...,False,help girl high school tour scare,classic spring quarter sunday woke random bed ...
6561,[Artsweek],Cage the Elephant at the Santa Barbara Bowl: A...,2024-07-16,11:53:00,[Lauren Chiou],https://dailynexus.com/2024-07-16/cage-the-ele...,American rock band Cage the Elephant performed...,False,cage elephant santa barbara bowl electrifying ...,american rock band cage elephant performed san...
6562,[Artsweek],"""Mustard Man"": The ""post coming-of-age-story""...",2024-07-16,16:05:00,[Kyra Schimpf],https://dailynexus.com/2024-07-16/mustard-man-...,"Courtesy of ""Mustard Man"" crew Fresh off its C...",False,mustard man post coming age story music mayhem...,courtesy mustard man crew fresh california pre...
6563,[On The Menu],Strawberry Avocado Toast: A Summer Delight,2024-07-18,10:00:00,[Pihu Jain],https://dailynexus.com/2024-07-18/117146/,"A perfect blend of the sweet and savory, this ...",False,strawberry avocado toast summer delight,perfect blend sweet savory recipe sure delight...


In [213]:
# Load data
nlp_df = pd.read_csv('data_storage/nlp_df.csv')
nlp_df['Authors'] = nlp_df['Authors'].apply(clean_authors)
nlp_df['Section'] = nlp_df['Section'].apply(clean_authors)
translated_df = pd.read_csv('scraped_files/translated-la-vista.csv')

# delete all rows after 115
translated_df = translated_df.drop(translated_df.index[116:])
translated_df.to_csv('data_storage/temp_translated-la-vista.csv', index=False)

# Merge the two DataFrames on Title but drop Section_y and change all NaN to None and rename Section_x to section
nlp_df = nlp_df.merge(translated_df, on='Title', how='left')

# Fix merging errors -  merge the two DataFrames on Title  and content- with manual index merging
# Merge nlp_df 5590 with translated_df 47, nlp_df 5594 with translated_df 49, nlp_df 5839 with translated_df 75, nlp_df 5880 with translated_df 80, nlp_df 5929 with translated_df 88, nlp_df 6269 with translated_df 98

# Title
nlp_df.loc[5590, 'English_Title'] = translated_df.loc[47, 'English_Title']
nlp_df.loc[5594, 'English_Title'] = translated_df.loc[49, 'English_Title']
nlp_df.loc[5839, 'English_Title'] = translated_df.loc[75, 'English_Title']
nlp_df.loc[5880, 'English_Title'] = translated_df.loc[80, 'English_Title']
nlp_df.loc[5929, 'English_Title'] = translated_df.loc[88, 'English_Title']
nlp_df.loc[6269, 'English_Title'] = translated_df.loc[98, 'English_Title']

# Content
nlp_df.loc[5590, 'Translated_Content'] = translated_df.loc[47, 'Translated_Content']
nlp_df.loc[5594, 'Translated_Content'] = translated_df.loc[49, 'Translated_Content']
nlp_df.loc[5839, 'Translated_Content'] = translated_df.loc[75, 'Translated_Content']
nlp_df.loc[5880, 'Translated_Content'] = translated_df.loc[80, 'Translated_Content']
nlp_df.loc[5929, 'Translated_Content'] = translated_df.loc[88, 'Translated_Content']
nlp_df.loc[6269, 'Translated_Content'] = translated_df.loc[98, 'Translated_Content']


nlp_df.drop(columns=['Section_y'], inplace=True)
nlp_df.fillna('None', inplace=True)
nlp_df.rename(columns={'Section_x': 'Section'}, inplace=True)

# Save to a new CSV file
nlp_df.to_csv('data_storage/translated_nlp_df.csv', index=False)

nlp_df

,Section,Title,Date,Time,Authors,URL,Content,is_spanish,Processed_Title,Processed_Content,English_Title,Translated_Content
0,[Sports],UCSB End Non-Conf. Portion of Season in Winnin...,2018-01-01,00:37:00,[Spencer Ault],https://dailynexus.com/2018-01-01/80339/,"After breaking through the defense, Jalen Cant...",False,ucsb end non conf portion season winning fashion,breaking defense jalen canty prepares shoot du...,None,None
1,[Sports],Gauchos to Host Long Beach and Cal Poly to Ope...,2018-01-02,00:47:00,[Calvin Thrift],https://dailynexus.com/2018-01-02/gauchos-to-h...,"Sarah Porter dribbles the ball, looking for a ...",False,gaucho host long beach cal poly open big west ...,sarah porter dribble ball looking pas non conf...,None,None
2,[Sports],Gauchos Outlast 49ers in Conference Opener,2018-01-05,19:30:00,[Calvin Thrift],https://dailynexus.com/2018-01-05/gauchos-outl...,Sarah Porter brings the ball up the court. Des...,False,gaucho outlast conference opener,sarah porter brings ball court despite another...,None,None
3,[News],Earthquake Measuring 3.8 Reported Near Isla Vi...,2018-01-06,19:49:00,[Hannah Jackson],https://dailynexus.com/2018-01-06/earthquake-m...,A 3.8 magnitude earthquake was reported off th...,False,earthquake measuring reported near isla vista ...,magnitude earthquake reported coast isla vista...,None,None
4,[Artsweek],The Friday Review: Saturation III by Brockhampton,2018-01-06,15:14:00,[Konrad Neithercutt],https://dailynexus.com/2018-01-06/the-friday-r...,The Internet's favorite boy band just solidifi...,False,friday review saturation iii brockhampton,internet favorite boy band solidified hip hop ...,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...
6560,[Daily Stench],Help! The girls on the high school tours scare me,2024-07-14,04:20:00,[Joseph R. Biden],https://dailynexus.com/2024-07-14/help-the-gir...,It was a classic spring quarter Sunday: I woke...,False,help girl high school tour scare,classic spring quarter sunday woke random bed ...,None,None
6561,[Artsweek],Cage the Elephant at the Santa Barbara Bowl: A...,2024-07-16,11:53:00,[Lauren Chiou],https://dailynexus.com/2024-07-16/cage-the-ele...,American rock band Cage the Elephant performed...,False,cage elephant santa barbara bowl electrifying ...,american rock band cage elephant performed san...,None,None
6562,[Artsweek],"""Mustard Man"": The ""post coming-of-age-story""...",2024-07-16,16:05:00,[Kyra Schimpf],https://dailynexus.com/2024-07-16/mustard-man-...,"Courtesy of ""Mustard Man"" crew Fresh off its C...",False,mustard man post coming age story music mayhem...,courtesy mustard man crew fresh california pre...,None,None
6563,[On The Menu],Strawberry Avocado Toast: A Summer Delight,2024-07-18,10:00:00,[Pihu Jain],https://dailynexus.com/2024-07-18/117146/,"A perfect blend of the sweet and savory, this ...",False,strawberry avocado toast summer delight,perfect blend sweet savory recipe sure delight...,None,None


## VADER Sentiment Analysis

In [275]:
# Function to analyze sentiment
def analyze_sentiment_vader(text):
    return analyzer.polarity_scores(text)

# Function to classify sentiment
def classify_sentiment(compound):
    if compound >= 0.05:
        return 'Positive'
    elif compound <= -0.05:
        return 'Negative'
    else:
        return 'Neutral'

In [279]:
# Load data
sentiment_df = pd.read_csv('data_storage/translated_nlp_df.csv')
sentiment_df['Authors'] = sentiment_df['Authors'].apply(clean_authors)
sentiment_df['Section'] = sentiment_df['Section'].apply(clean_authors)

# Initialize the VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

# Apply the sentiment analysis to the original 'Title' and 'Content'
tqdm.pandas()

# Apply sentiment analysis for Titles and Content
sentiment_df['Title_Sentiment'] = sentiment_df.progress_apply(
    lambda x: analyze_sentiment_vader(x['English_Title']) if x['is_spanish'] else analyze_sentiment_vader(x['Title']), axis=1)

sentiment_df['Content_Sentiment'] = sentiment_df.progress_apply(
    lambda x: analyze_sentiment_vader(x['Translated_Content']) if x['is_spanish'] else analyze_sentiment_vader(x['Content']), axis=1)

# Extract the sentiment scores into separate columns for Titles
sentiment_df[['Title_Neg', 'Title_Neu', 'Title_Pos', 'Title_Compound']] = sentiment_df['Title_Sentiment'].apply(pd.Series)

# Extract the sentiment scores into separate columns for Content
sentiment_df[['Content_Neg', 'Content_Neu', 'Content_Pos', 'Content_Compound']] = sentiment_df['Content_Sentiment'].apply(pd.Series)

# Drop the intermediate sentiment columns
sentiment_df.drop(columns=['Title_Sentiment', 'Content_Sentiment'], inplace=True)

sentiment_df

100%|██████████| 6565/6565 [03:22<00:00, 32.48it/s]


,Section,Title,Date,Time,Authors,URL,Content,is_spanish,Processed_Title,Processed_Content,English_Title,Translated_Content,Title_Neg,Title_Neu,Title_Pos,Title_Compound,Content_Neg,Content_Neu,Content_Pos,Content_Compound
0,[Sports],UCSB End Non-Conf. Portion of Season in Winnin...,2018-01-01,00:37:00,[Spencer Ault],https://dailynexus.com/2018-01-01/80339/,"After breaking through the defense, Jalen Cant...",False,ucsb end non conf portion season winning fashion,breaking defense jalen canty prepares shoot du...,None,None,0.000,0.702,0.298,0.5267,0.048,0.859,0.093,0.9755
1,[Sports],Gauchos to Host Long Beach and Cal Poly to Ope...,2018-01-02,00:47:00,[Calvin Thrift],https://dailynexus.com/2018-01-02/gauchos-to-h...,"Sarah Porter dribbles the ball, looking for a ...",False,gaucho host long beach cal poly open big west ...,sarah porter dribble ball looking pas non conf...,None,None,0.000,0.833,0.167,0.3400,0.028,0.859,0.113,0.9952
2,[Sports],Gauchos Outlast 49ers in Conference Opener,2018-01-05,19:30:00,[Calvin Thrift],https://dailynexus.com/2018-01-05/gauchos-outl...,Sarah Porter brings the ball up the court. Des...,False,gaucho outlast conference opener,sarah porter brings ball court despite another...,None,None,0.000,1.000,0.000,0.0000,0.062,0.823,0.115,0.9957
3,[News],Earthquake Measuring 3.8 Reported Near Isla Vi...,2018-01-06,19:49:00,[Hannah Jackson],https://dailynexus.com/2018-01-06/earthquake-m...,A 3.8 magnitude earthquake was reported off th...,False,earthquake measuring reported near isla vista ...,magnitude earthquake reported coast isla vista...,None,None,0.000,1.000,0.000,0.0000,0.000,0.980,0.020,0.3919
4,[Artsweek],The Friday Review: Saturation III by Brockhampton,2018-01-06,15:14:00,[Konrad Neithercutt],https://dailynexus.com/2018-01-06/the-friday-r...,The Internet's favorite boy band just solidifi...,False,friday review saturation iii brockhampton,internet favorite boy band solidified hip hop ...,None,None,0.000,1.000,0.000,0.0000,0.054,0.838,0.109,0.9944
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6560,[Daily Stench],Help! The girls on the high school tours scare me,2024-07-14,04:20:00,[Joseph R. Biden],https://dailynexus.com/2024-07-14/help-the-gir...,It was a classic spring quarter Sunday: I woke...,False,help girl high school tour scare,classic spring quarter sunday woke random bed ...,None,None,0.246,0.564,0.190,-0.2003,0.132,0.772,0.097,-0.9834
6561,[Artsweek],Cage the Elephant at the Santa Barbara Bowl: A...,2024-07-16,11:53:00,[Lauren Chiou],https://dailynexus.com/2024-07-16/cage-the-ele...,American rock band Cage the Elephant performed...,False,cage elephant santa barbara bowl electrifying ...,american rock band cage elephant performed san...,None,None,0.000,1.000,0.000,0.0000,0.044,0.831,0.125,0.9987
6562,[Artsweek],"""Mustard Man"": The ""post coming-of-age-story""...",2024-07-16,16:05:00,[Kyra Schimpf],https://dailynexus.com/2024-07-16/mustard-man-...,"Courtesy of ""Mustard Man"" crew Fresh off its C...",False,mustard man post coming age story music mayhem...,courtesy mustard man crew fresh california pre...,None,None,0.000,1.000,0.000,0.0000,0.028,0.846,0.126,0.9998
6563,[On The Menu],Strawberry Avocado Toast: A Summer Delight,2024-07-18,10:00:00,[Pihu Jain],https://dailynexus.com/2024-07-18/117146/,"A perfect blend of the sweet and savory, this ...",False,strawberry avocado toast summer delight,perfect blend sweet savory recipe sure delight...,None,None,0.000,0.562,0.438,0.5994,0.010,0.796,0.195,0.9966


In [280]:
# Use compound scores to classify overall sentiment of Title and Content
sentiment_df['Title_Sentiment'] = sentiment_df['Title_Compound'].apply(classify_sentiment)
sentiment_df['Content_Sentiment'] = sentiment_df['Content_Compound'].apply(classify_sentiment)

# Save the updated dataframe to a new CSV file
sentiment_df.to_csv('data_storage/original_sentiment.csv', index=False)

sentiment_df

,Section,Title,Date,Time,Authors,URL,Content,is_spanish,Processed_Title,Processed_Content,English_Title,Translated_Content,Title_Neg,Title_Neu,Title_Pos,Title_Compound,Content_Neg,Content_Neu,Content_Pos,Content_Compound,Title_Sentiment,Content_Sentiment
0,[Sports],UCSB End Non-Conf. Portion of Season in Winnin...,2018-01-01,00:37:00,[Spencer Ault],https://dailynexus.com/2018-01-01/80339/,"After breaking through the defense, Jalen Cant...",False,ucsb end non conf portion season winning fashion,breaking defense jalen canty prepares shoot du...,None,None,0.000,0.702,0.298,0.5267,0.048,0.859,0.093,0.9755,Positive,Positive
1,[Sports],Gauchos to Host Long Beach and Cal Poly to Ope...,2018-01-02,00:47:00,[Calvin Thrift],https://dailynexus.com/2018-01-02/gauchos-to-h...,"Sarah Porter dribbles the ball, looking for a ...",False,gaucho host long beach cal poly open big west ...,sarah porter dribble ball looking pas non conf...,None,None,0.000,0.833,0.167,0.3400,0.028,0.859,0.113,0.9952,Positive,Positive
2,[Sports],Gauchos Outlast 49ers in Conference Opener,2018-01-05,19:30:00,[Calvin Thrift],https://dailynexus.com/2018-01-05/gauchos-outl...,Sarah Porter brings the ball up the court. Des...,False,gaucho outlast conference opener,sarah porter brings ball court despite another...,None,None,0.000,1.000,0.000,0.0000,0.062,0.823,0.115,0.9957,Neutral,Positive
3,[News],Earthquake Measuring 3.8 Reported Near Isla Vi...,2018-01-06,19:49:00,[Hannah Jackson],https://dailynexus.com/2018-01-06/earthquake-m...,A 3.8 magnitude earthquake was reported off th...,False,earthquake measuring reported near isla vista ...,magnitude earthquake reported coast isla vista...,None,None,0.000,1.000,0.000,0.0000,0.000,0.980,0.020,0.3919,Neutral,Positive
4,[Artsweek],The Friday Review: Saturation III by Brockhampton,2018-01-06,15:14:00,[Konrad Neithercutt],https://dailynexus.com/2018-01-06/the-friday-r...,The Internet's favorite boy band just solidifi...,False,friday review saturation iii brockhampton,internet favorite boy band solidified hip hop ...,None,None,0.000,1.000,0.000,0.0000,0.054,0.838,0.109,0.9944,Neutral,Positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6560,[Daily Stench],Help! The girls on the high school tours scare me,2024-07-14,04:20:00,[Joseph R. Biden],https://dailynexus.com/2024-07-14/help-the-gir...,It was a classic spring quarter Sunday: I woke...,False,help girl high school tour scare,classic spring quarter sunday woke random bed ...,None,None,0.246,0.564,0.190,-0.2003,0.132,0.772,0.097,-0.9834,Negative,Negative
6561,[Artsweek],Cage the Elephant at the Santa Barbara Bowl: A...,2024-07-16,11:53:00,[Lauren Chiou],https://dailynexus.com/2024-07-16/cage-the-ele...,American rock band Cage the Elephant performed...,False,cage elephant santa barbara bowl electrifying ...,american rock band cage elephant performed san...,None,None,0.000,1.000,0.000,0.0000,0.044,0.831,0.125,0.9987,Neutral,Positive
6562,[Artsweek],"""Mustard Man"": The ""post coming-of-age-story""...",2024-07-16,16:05:00,[Kyra Schimpf],https://dailynexus.com/2024-07-16/mustard-man-...,"Courtesy of ""Mustard Man"" crew Fresh off its C...",False,mustard man post coming age story music mayhem...,courtesy mustard man crew fresh california pre...,None,None,0.000,1.000,0.000,0.0000,0.028,0.846,0.126,0.9998,Neutral,Positive
6563,[On The Menu],Strawberry Avocado Toast: A Summer Delight,2024-07-18,10:00:00,[Pihu Jain],https://dailynexus.com/2024-07-18/117146/,"A perfect blend of the sweet and savory, this ...",False,strawberry avocado toast summer delight,perfect blend sweet savory recipe sure delight...,None,None,0.000,0.562,0.438,0.5994,0.010,0.796,0.195,0.9966,Positive,Positive


In [223]:
sentiment_df['Content_Sentiment'].value_counts()

Positive    5293
Negative    1192
Neutral       80
Name: Content_Sentiment, dtype: int64

## NRC Emotion Lexicon Analysis

In [226]:
# Function to analyze emotions using NRC Lexicon
def analyze_emotions(text, lang='en'):
    emotion = NRCLex(text, language=lang)
    emotion_scores = emotion.affect_frequencies
    return emotion_scores

In [321]:
# Load the data
emotion_df = pd.read_csv('data_storage/original_sentiment.csv')
emotion_df['Authors'] = emotion_df['Authors'].apply(clean_authors)
emotion_df['Section'] = emotion_df['Section'].apply(clean_authors)

emolex_df = pd.read_csv("NRC-Emotion-Lexicon/OneFilePerLanguage/Spanish-NRC-EmoLex.txt", sep='\t', keep_default_na=False)
emoint_df = pd.read_csv('NRC-Emotion-Intensity-Lexicon/OneFilePerLanguage/Spanish-NRC-Emotion-Intensity-Lexicon-v1.txt', sep='\t', keep_default_na=False)
emovad_df = pd.read_csv("NRC-VAD-Lexicon/OneFilePerLanguage/Spanish-NRC-VAD-Lexicon.txt", sep='\t', keep_default_na=False)

emolex_df

,English Word,anger,anticipation,disgust,fear,joy,negative,positive,sadness,surprise,trust,Spanish Word
0,aback,0,0,0,0,0,0,0,0,0,0,detrás
1,abacus,0,0,0,0,0,0,0,0,0,1,ábaco
2,abandon,0,0,0,1,0,1,0,1,0,0,abandonar
3,abandoned,1,0,0,1,0,1,0,1,0,0,abandonado
4,abandonment,1,0,0,1,0,1,0,1,1,0,abandono
...,...,...,...,...,...,...,...,...,...,...,...,...
14149,zone,0,0,0,0,0,0,0,0,0,0,zona
14150,zoo,0,0,0,0,0,0,0,0,0,0,zoo
14151,zoological,0,0,0,0,0,0,0,0,0,0,zoológico
14152,zoology,0,0,0,0,0,0,0,0,0,0,zoología


In [319]:
all_emotions = ['anger', 'anticipation', 'disgust', 'fear', 'joy', 'negative', 'positive', 'sadness', 'surprise', 'trust']  

# Function to get emotion scores from the EmoLex
def get_emolex_scores(word, is_spanish):
    if is_spanish:
        scores = emolex_df[emolex_df['Spanish Word'].str.lower() == word.lower()]
    else:
        scores = emolex_df[emolex_df['English Word'].str.lower() == word.lower()]
    if not scores.empty:
        return scores.iloc[0].to_dict()
    return None

# Function to get emotion intensity scores from the EmoInt
def get_emoint_scores(word, is_spanish):
    if is_spanish:
        scores = emoint_df[emoint_df['Spanish Word'].str.lower() == word.lower()]
    else:
        scores = emoint_df[emoint_df['English Word'].str.lower() == word.lower()]
    if not scores.empty:
        return scores.iloc[0].to_dict()
    return None

# Function to get VAD scores from the EmoVAD
def get_emovad_scores(word, is_spanish):
    if is_spanish:
        scores = emovad_df[emovad_df['Spanish Word'].str.lower() == word.lower()] 
    else:
        scores = emovad_df[emovad_df['English Word'].str.lower() == word.lower()]
    if not scores.empty:
        return scores.iloc[0].to_dict()
    return None

# Fisher's Z transformation
def fisher_z(r):
    return np.arctanh(r)

# Inverse Fisher's Z transformation
def inverse_fisher_z(z):
    return np.tanh(z)

# Function to aggregate scores for a list of words
def aggregate_scores(words, is_spanish):
    # Initialize all emotions with 0
    emotion_scores = {emotion: 0 for emotion in all_emotions}
    intensity_scores = {emotion: 0 for emotion in all_emotions}
    vad_scores = {'Valence': 0.0, 'Arousal': 0.0, 'Dominance': 0.0}
    vad_count = 0
    emolex_count = 0
    emoint_count = 0
    total_word_count = len(words)
    for word in words:
        emolex = get_emolex_scores(word, is_spanish)
        emoint = get_emoint_scores(word, is_spanish)
        emovad = get_emovad_scores(word, is_spanish)
        
        if emolex:
            emolex_count += 1
            for emotion, score in emolex.items():
                if emotion in all_emotions:
                    emotion_scores[emotion] += score

        if emoint:
            emoint_count += 1
            if emoint['Emotion'] in all_emotions:
                intensity_scores[emoint['Emotion']] += emoint['Emotion-Intensity-Score']

        if emovad:
            vad_scores['Valence'] += emovad['Valence']
            vad_scores['Arousal'] += emovad['Arousal']
            vad_scores['Dominance'] += emovad['Dominance']
            vad_count += 1
    # Ensure no NaN values are present in the calculations
    emotion_scores = {k: 0 if pd.isna(v) else v for k, v in emotion_scores.items()}
    intensity_scores = {k: 0 if pd.isna(v) else v for k, v in intensity_scores.items()}
    vad_scores = {k: 0.0 if pd.isna(v) else v for k, v in vad_scores.items()}

    # Calculate averages for VAD and EmoInt using Fisher's Z transformation
    if vad_count > 0:
        vad_scores_z = {k: fisher_z(vad_scores[k] / vad_count) for k in vad_scores}
        vad_scores_avg = {k: inverse_fisher_z(v) for k, v in vad_scores_z.items()}
    else:
        vad_scores_avg = {k: 0.0 for k in vad_scores}

    if emoint_count > 0:
        intensity_scores_z = {k: fisher_z(intensity_scores[k] / emoint_count) for k in intensity_scores}
        intensity_scores_avg = {k: inverse_fisher_z(v) for k, v in intensity_scores_z.items()}
    else:
        intensity_scores_avg = {k: 0.0 for k in intensity_scores}

    # Calculate averages over total words using Fisher's Z transformation
    vad_scores_total_z = {k: fisher_z(vad_scores[k] / total_word_count) for k in vad_scores}
    vad_scores_total_avg = {f"{k}_total_avg": inverse_fisher_z(v) for k, v in vad_scores_total_z.items()}

    intensity_scores_total_z = {k: fisher_z(intensity_scores[k] / total_word_count) for k in intensity_scores}
    intensity_scores_total_avg = {f"{k}_total_avg": inverse_fisher_z(v) for k, v in intensity_scores_total_z.items()}

    emotion_scores_total_avg = {f"{k}_total_avg": emotion_scores[k] / total_word_count for k in emotion_scores}
    
    return (
        emotion_scores, intensity_scores, vad_scores_avg, 
        emolex_count, total_word_count, 
        vad_scores_total_avg, intensity_scores_total_avg, emotion_scores_total_avg
    )

# Add a progress bar
tqdm.pandas()

# Apply aggregation to Processed_Title only
(
    emotion_df['Title_Emotion_Scores'],
    emotion_df['Title_Intensity_Scores'],
    emotion_df['Title_VAD_Scores'],
    emotion_df['Title_EmoLex_Words'],
    emotion_df['Title_Total_Words'],
    emotion_df['Title_VAD_Total_Avg'],
    emotion_df['Title_Intensity_Total_Avg'],
    emotion_df['Title_Emotion_Total_Avg']
) = zip(*emotion_df.progress_apply(lambda x: aggregate_scores(x['Processed_Title'].split(), x['is_spanish']), axis=1))

# Expand Title_Emotion_Scores into separate columns
emotion_df = pd.concat([emotion_df.drop(['Title_Emotion_Scores'], axis=1), emotion_df['Title_Emotion_Scores'].progress_apply(pd.Series)], axis=1)

# Expand Title_Intensity_Scores into separate columns and add '_intensity' suffix
intensity_scores_df = emotion_df['Title_Intensity_Scores'].progress_apply(pd.Series).add_suffix('_intensity')
emotion_df = pd.concat([emotion_df.drop(['Title_Intensity_Scores'], axis=1), intensity_scores_df], axis=1)

# Expand Title_VAD_Scores into separate columns
vad_scores_df = emotion_df['Title_VAD_Scores'].progress_apply(pd.Series)
emotion_df = pd.concat([emotion_df.drop(['Title_VAD_Scores'], axis=1), vad_scores_df], axis=1)

# Expand Total Averages for VAD, Intensity, and Emotion counts
vad_total_avg_df = emotion_df['Title_VAD_Total_Avg'].progress_apply(pd.Series)
emotion_df = pd.concat([emotion_df.drop(['Title_VAD_Total_Avg'], axis=1), vad_total_avg_df], axis=1)

intensity_total_avg_df = emotion_df['Title_Intensity_Total_Avg'].progress_apply(pd.Series)
emotion_df = pd.concat([emotion_df.drop(['Title_Intensity_Total_Avg'], axis=1), intensity_total_avg_df], axis=1)

emotion_total_avg_df = emotion_df['Title_Emotion_Total_Avg'].progress_apply(pd.Series)
emotion_df = pd.concat([emotion_df.drop(['Title_Emotion_Total_Avg'], axis=1), emotion_total_avg_df], axis=1)

# Save the updated dataframe to a new CSV file
emotion_df.to_csv('data_storage/full_sentiment.csv', index=False)

emotion_df

100%|██████████| 6565/6565 [00:01<00:00, 5372.81it/s]


,Section,Title,Date,Time,Authors,URL,Content,is_spanish,Processed_Title,Processed_Content,English_Title,Translated_Content,Title_Pos,Title_Neu,Title_Neg,Title_Compound,Content_Pos,Content_Neu,Content_Neg,Content_Compound,Title_Sentiment,Content_Sentiment,Title_EmoLex_Words,Title_Total_Words,anger,anticipation,disgust,fear,joy,negative,positive,sadness,surprise,trust,anger_intensity,anticipation_intensity,disgust_intensity,fear_intensity,joy_intensity,negative_intensity,positive_intensity,sadness_intensity,surprise_intensity,trust_intensity,Valence,Arousal,Dominance,Valence_total_avg,Arousal_total_avg,Dominance_total_avg,anger_total_avg,anticipation_total_avg,disgust_total_avg,fear_total_avg,joy_total_avg,negative_total_avg,positive_total_avg,sadness_total_avg,surprise_total_avg,trust_total_avg,anger_total_avg,anticipation_total_avg,disgust_total_avg,fear_total_avg,joy_total_avg,negative_total_avg,positive_total_avg,sadness_total_avg,surprise_total_avg,trust_total_avg
0,[Sports],UCSB End Non-Conf. Portion of Season in Winnin...,2018-01-01,00:37:00,[Spencer Ault],https://dailynexus.com/2018-01-01/80339/,"After breaking through the defense, Jalen Cant...",False,ucsb end non conf portion season winning fashion,breaking defense jalen canty prepares shoot du...,None,None,0.000,0.702,0.298,0.5267,0.048,0.859,0.093,0.9755,Positive,Positive,5,8,0,1,1,0,1,0,1,1,1,1,0.000,0.664,0.0,0.000,0.000,0.0,0.0,0.000,0.0,0.266,0.661600,0.490200,0.555200,0.413500,0.306375,0.347000,0.000000,0.083000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.03325,0.000000,0.125000,0.125,0.000000,0.125000,0.000000,0.125000,0.125000,0.125000,0.125000
1,[Sports],Gauchos to Host Long Beach and Cal Poly to Ope...,2018-01-02,00:47:00,[Calvin Thrift],https://dailynexus.com/2018-01-02/gauchos-to-h...,"Sarah Porter dribbles the ball, looking for a ...",False,gaucho host long beach cal poly open big west ...,sarah porter dribble ball looking pas non conf...,None,None,0.000,0.833,0.167,0.3400,0.028,0.859,0.113,0.9952,Positive,Positive,5,10,0,1,0,0,1,0,0,0,0,0,0.000,0.406,0.0,0.000,0.500,0.0,0.0,0.000,0.0,0.000,0.702143,0.503286,0.526857,0.491500,0.352300,0.368800,0.000000,0.040600,0.0,0.000000,0.050000,0.0,0.0,0.000000,0.0,0.00000,0.000000,0.100000,0.000,0.000000,0.100000,0.000000,0.000000,0.000000,0.000000,0.000000
2,[Sports],Gauchos Outlast 49ers in Conference Opener,2018-01-05,19:30:00,[Calvin Thrift],https://dailynexus.com/2018-01-05/gauchos-outl...,Sarah Porter brings the ball up the court. Des...,False,gaucho outlast conference opener,sarah porter brings ball court despite another...,None,None,0.000,1.000,0.000,0.0000,0.062,0.823,0.115,0.9957,Neutral,Positive,3,4,0,0,0,0,0,0,0,0,0,0,0.000,0.000,0.0,0.000,0.000,0.0,0.0,0.000,0.0,0.000,0.630667,0.642333,0.556000,0.473000,0.481750,0.417000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.00000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,[News],Earthquake Measuring 3.8 Reported Near Isla Vi...,2018-01-06,19:49:00,[Hannah Jackson],https://dailynexus.com/2018-01-06/earthquake-m...,A 3.8 magnitude earthquake was reported off th...,False,earthquake measuring reported near isla vista ...,magnitude earthquake reported coast isla vista...,None,None,0.000,1.000,0.000,0.0000,0.000,0.980,0.020,0.3919,Neutral,Positive,5,7,1,0,0,1,0,1,1,1,1,0,0.391,0.422,0.0,0.000,0.000,0.0,0.0,0.000,0.0,0.000,0.471200,0.499000,0.563200,0.336571,0.356429,0.402286,0.055857,0.060286,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.00000,0.142857,0.000000,0.000,0.142857,0.000000,0.142857,0.142857,0.142857,0.142857,0.000000
4,[Artsweek],The Friday Review: Saturation III by Brockhampton,2018-01-06,15:14:00,[Konrad Neithercutt],https://dailynexus.com/2018-01-06/the-friday-r...,The Internet's favorite boy band just solidifi...,False,friday review saturation iii brockhampton,internet favorite boy band solidified hip hop ...,None,None,0.000,1.000,0.000,0.0000,0.054,0.838,0.109,0.9944,Neutral,Positive,2,5,

In [322]:
# Load the data
emotion_df = pd.read_csv('data_storage/full_sentiment.csv')
emotion_df['Authors'] = emotion_df['Authors'].apply(clean_authors)
emotion_df['Section'] = emotion_df['Section'].apply(clean_authors)

# Function to aggregate scores for a list of words from content
def aggregate_content_scores(words, is_spanish):
    # Initialize all emotions with 0
    emotion_scores = {emotion: 0 for emotion in all_emotions}
    intensity_scores = {emotion: 0 for emotion in all_emotions}
    vad_scores = {'Valence': 0.0, 'Arousal': 0.0, 'Dominance': 0.0}
    vad_count = 0
    emolex_count = 0
    emoint_count = 0
    total_word_count = len(words)
    
    for word in words:
        emolex = get_emolex_scores(word, is_spanish)
        emoint = get_emoint_scores(word, is_spanish)
        emovad = get_emovad_scores(word, is_spanish)
        
        if emolex:
            emolex_count += 1
            for emotion, score in emolex.items():
                if emotion in all_emotions:
                    emotion_scores[emotion] += score

        if emoint:
            emoint_count += 1
            if emoint['Emotion'] in all_emotions:
                intensity_scores[emoint['Emotion']] += emoint['Emotion-Intensity-Score']

        if emovad:
            vad_scores['Valence'] += emovad['Valence']
            vad_scores['Arousal'] += emovad['Arousal']
            vad_scores['Dominance'] += emovad['Dominance']
            vad_count += 1

    # Ensure no NaN values are present in the calculations
    emotion_scores = {k: 0 if pd.isna(v) else v for k, v in emotion_scores.items()}
    intensity_scores = {k: 0 if pd.isna(v) else v for k, v in intensity_scores.items()}
    vad_scores = {k: 0.0 if pd.isna(v) else v for k, v in vad_scores.items()}

    # Calculate averages for VAD and EmoInt using Fisher's Z transformation
    if vad_count > 0:
        vad_scores_z = {k: fisher_z(vad_scores[k] / vad_count) for k in vad_scores}
        vad_scores_avg = {k: inverse_fisher_z(v) for k, v in vad_scores_z.items()}
    else:
        vad_scores_avg = {k: 0.0 for k in vad_scores}

    if emoint_count > 0:
        intensity_scores_z = {k: fisher_z(intensity_scores[k] / emoint_count) for k in intensity_scores}
        intensity_scores_avg = {k: inverse_fisher_z(v) for k, v in intensity_scores_z.items()}
    else:
        intensity_scores_avg = {k: 0.0 for k in intensity_scores}

    # Calculate averages over total words using Fisher's Z transformation
    vad_scores_total_z = {k: fisher_z(vad_scores[k] / total_word_count) for k in vad_scores}
    vad_scores_total_avg = {f"{k}_total_avg": inverse_fisher_z(v) for k, v in vad_scores_total_z.items()}

    intensity_scores_total_z = {k: fisher_z(intensity_scores[k] / total_word_count) for k in intensity_scores}
    intensity_scores_total_avg = {f"{k}_total_avg": inverse_fisher_z(v) for k, v in intensity_scores_total_z.items()}

    emotion_scores_total_avg = {f"{k}_total_avg": emotion_scores[k] / total_word_count for k in emotion_scores}
    
    return (
        emotion_scores, intensity_scores, vad_scores_avg, 
        emolex_count, total_word_count, 
        vad_scores_total_avg, intensity_scores_total_avg, emotion_scores_total_avg
    )

# Apply aggregation to Processed_Content
(
    emotion_df['Content_Emotion_Scores'],
    emotion_df['Content_Intensity_Scores'],
    emotion_df['Content_VAD_Scores'],
    emotion_df['Content_EmoLex_Words'],
    emotion_df['Content_Total_Words'],
    emotion_df['Content_VAD_Total_Avg'],
    emotion_df['Content_Intensity_Total_Avg'],
    emotion_df['Content_Emotion_Total_Avg']
) = zip(*emotion_df.progress_apply(lambda x: aggregate_content_scores(x['Processed_Content'].split(), x['is_spanish']), axis=1))

# Expand Content_Emotion_Scores into separate columns
emotion_df = pd.concat([emotion_df.drop(['Content_Emotion_Scores'], axis=1), emotion_df['Content_Emotion_Scores'].progress_apply(pd.Series)], axis=1)

# Expand Content_Intensity_Scores into separate columns and add '_intensity' suffix
intensity_scores_df = emotion_df['Content_Intensity_Scores'].progress_apply(pd.Series).add_suffix('_intensity')
emotion_df = pd.concat([emotion_df.drop(['Content_Intensity_Scores'], axis=1), intensity_scores_df], axis=1)

# Expand Content_VAD_Scores into separate columns
vad_scores_df = emotion_df['Content_VAD_Scores'].progress_apply(pd.Series)
emotion_df = pd.concat([emotion_df.drop(['Content_VAD_Scores'], axis=1), vad_scores_df], axis=1)

# Expand Total Averages for VAD, Intensity, and Emotion counts
vad_total_avg_df = emotion_df['Content_VAD_Total_Avg'].progress_apply(pd.Series)
emotion_df = pd.concat([emotion_df.drop(['Content_VAD_Total_Avg'], axis=1), vad_total_avg_df], axis=1)

intensity_total_avg_df = emotion_df['Content_Intensity_Total_Avg'].progress_apply(pd.Series)
emotion_df = pd.concat([emotion_df.drop(['Content_Intensity_Total_Avg'], axis=1), intensity_total_avg_df], axis=1)

emotion_total_avg_df = emotion_df['Content_Emotion_Total_Avg'].progress_apply(pd.Series)
emotion_df = pd.concat([emotion_df.drop(['Content_Emotion_Total_Avg'], axis=1), emotion_total_avg_df], axis=1)

# Save the updated dataframe to a new CSV file
emotion_df.to_csv('data_storage/final_full_sentiment.csv', index=False)

emotion_df

100%|██████████| 6565/6565 [00:01<00:00, 5361.73it/s]


,Section,Title,Date,Time,Authors,URL,Content,is_spanish,Processed_Title,Processed_Content,English_Title,Translated_Content,Title_Pos,Title_Neu,Title_Neg,Title_Compound,Content_Pos,Content_Neu,Content_Neg,Content_Compound,Title_Sentiment,Content_Sentiment,Title_EmoLex_Words,Title_Total_Words,anger,anticipation,disgust,fear,joy,negative,positive,sadness,surprise,trust,anger_intensity,anticipation_intensity,disgust_intensity,fear_intensity,joy_intensity,negative_intensity,positive_intensity,sadness_intensity,surprise_intensity,trust_intensity,Valence,Arousal,Dominance,Valence_total_avg,Arousal_total_avg,Dominance_total_avg,anger_total_avg,anticipation_total_avg,disgust_total_avg,fear_total_avg,joy_total_avg,negative_total_avg,positive_total_avg,sadness_total_avg,surprise_total_avg,trust_total_avg,anger_total_avg.1,anticipation_total_avg.1,disgust_total_avg.1,fear_total_avg.1,joy_total_avg.1,negative_total_avg.1,positive_total_avg.1,sadness_total_avg.1,surprise_total_avg.1,trust_total_avg.1,Content_EmoLex_Words,Content_Total_Words,anger,anticipation,disgust,fear,joy,negative,positive,sadness,surprise,trust,anger_intensity,anticipation_intensity,disgust_intensity,fear_intensity,joy_intensity,negative_intensity,positive_intensity,sadness_intensity,surprise_intensity,trust_intensity,Valence,Arousal,Dominance,Valence_total_avg,Arousal_total_avg,Dominance_total_avg,anger_total_avg,anticipation_total_avg,disgust_total_avg,fear_total_avg,joy_total_avg,negative_total_avg,positive_total_avg,sadness_total_avg,surprise_total_avg,trust_total_avg,anger_total_avg,anticipation_total_avg,disgust_total_avg,fear_total_avg,joy_total_avg,negative_total_avg,positive_total_avg,sadness_total_avg,surprise_total_avg,trust_total_avg
0,[Sports],UCSB End Non-Conf. Portion of Season in Winnin...,2018-01-01,00:37:00,[Spencer Ault],https://dailynexus.com/2018-01-01/80339/,"After breaking through the defense, Jalen Cant...",False,ucsb end non conf portion season winning fashion,breaking defense jalen canty prepares shoot du...,None,None,0.000,0.702,0.298,0.5267,0.048,0.859,0.093,0.9755,Positive,Positive,5,8,0,1,1,0,1,0,1,1,1,1,0.000,0.664,0.0,0.000,0.000,0.0,0.0,0.000,0.0,0.266,0.661600,0.490200,0.555200,0.413500,0.306375,0.347000,0.000000,0.083000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.03325,0.000000,0.125000,0.125,0.000000,0.125000,0.000000,0.125000,0.125000,0.125000,0.125000,178,377,16,12,3,14,5,24,28,8,3,12,7.943,4.593,0.695,0.640,4.944,0.0,0.0,1.022,0.516,5.867,0.593202,0.454537,0.521504,0.380782,0.291772,0.334759,0.021069,0.012183,0.001844,0.001698,0.013114,0.0,0.0,0.002711,0.001369,0.015562,0.042440,0.031830,0.007958,0.037135,0.013263,0.063660,0.074271,0.021220,0.007958,0.031830
1,[Sports],Gauchos to Host Long Beach and Cal Poly to Ope...,2018-01-02,00:47:00,[Calvin Thrift],https://dailynexus.com/2018-01-02/gauchos-to-h...,"Sarah Porter dribbles the ball, looking for a ...",False,gaucho host long beach cal poly open big west ...,sarah porter dribble ball looking pas non conf...,None,None,0.000,0.833,0.167,0.3400,0.028,0.859,0.113,0.9952,Positive,Positive,5,10,0,1,0,0,1,0,0,0,0,0,0.000,0.406,0.0,0.000,0.500,0.0,0.0,0.000,0.0,0.000,0.702143,0.503286,0.526857,0.491500,0.352300,0.368800,0.000000,0.040600,0.0,0.000000,0.050000,0.0,0.0,0.000000,0.0,0.00000,0.000000,0.100000,0.000,0.000000,0.100000,0.000000,0.000000,0.000000,0.000000,0.000000,181,339,12,23,5,8,15,14,29,8,7,21,4.877,11.848,0.000,0.500,8.441,0.0,0.0,0.681,0.000,6.609,0.619491,0.466806,0.535401,0.405684,0.305696,0.350617,0.014386,0.034950,0.000000,0.001475,0.024900,0.0,0.0,0.002009,0.000000,0.019496,0.035398,0.067847,0.014749,0.023599,0.044248,0.041298,0.085546,0.023599,0.020649,0.061947
2,[Sports],Gauchos Outlast 49ers in Conference Opener,2018-01-05,19:30:00,[Calvin Thrift],https://dailynexus.com/2018-01-05/gauchos-outl...,Sarah Porter brings the ball up the court. Des...,False,gaucho outlast conference opener,sarah porter brings ball court despite another...,None,None,0.000,1.000,0.000,0.0

In [333]:
emotion_df.columns = ["Section", "Title", "Date", "Time", "Authors", "URL", "Content", "is_spanish", "Processed_Title", "Processed_Content", "English_Title", "Translated_Content", "Title_Pos", "Title_Neu", "Title_Neg", "Title_Compound", "Content_Pos", "Content_Neu", "Content_Neg", "Content_Compound", "Title_Sentiment", "Content_Sentiment", "Title_EmoLex_Words", "Title_Total_Words", "Title_anger", "Title_anticipation", "Title_disgust", "Title_fear", "Title_joy", "Title_negative", "Title_positive", "Title_sadness", "Title_surprise", "Title_trust", "Title_anger_intensity", "Title_anticipation_intensity", "Title_disgust_intensity", "Title_fear_intensity", "Title_joy_intensity", "Title_negative_intensity", "Title_positive_intensity", "Title_sadness_intensity", "Title_surprise_intensity", "Title_trust_intensity", "Title_Valence", "Title_Arousal", "Title_Dominance", "Title_Valence_total_avg", "Title_Arousal_total_avg", "Title_Dominance_total_avg", "Title_intensity_anger_total_avg", "Title_intensity_anticipation_total_avg", "Title_intensity_disgust_total_avg", "Title_intensity_fear_total_avg", "Title_intensity_joy_total_avg", "Title_intensity_negative_total_avg", "Title_intensity_positive_total_avg", "Title_intensity_sadness_total_avg", "Title_intensity_surprise_total_avg", "Title_intensity_trust_total_avg", "Title_anger_total_avg", "Title_anticipation_total_avg", "Title_disgust_total_avg", "Title_fear_total_avg", "Title_joy_total_avg", "Title_negative_total_avg", "Title_positive_total_avg", "Title_sadness_total_avg", "Title_surprise_total_avg", "Title_trust_total_avg", "Content_EmoLex_Words", "Content_Total_Words", "Content_anger", "Content_anticipation", "Content_disgust", "Content_fear", "Content_joy", "Content_negative", "Content_positive", "Content_sadness", "Content_surprise", "Content_trust", "Content_anger_intensity", "Content_anticipation_intensity", "Content_disgust_intensity", "Content_fear_intensity", "Content_joy_intensity", "Content_negative_intensity", "Content_positive_intensity", "Content_sadness_intensity", "Content_surprise_intensity", "Content_trust_intensity", "Content_Valence", "Content_Arousal", "Content_Dominance", "Content_Valence_total_avg", "Content_Arousal_total_avg", "Content_Dominance_total_avg", "Content_intensity_anger_total_avg", "Content_intensity_anticipation_total_avg", "Content_intensity_disgust_total_avg", "Content_intensity_fear_total_avg", "Content_intensity_joy_total_avg", "Content_intensity_negative_total_avg", "Content_intensity_positive_total_avg", "Content_intensity_sadness_total_avg", "Content_intensity_surprise_total_avg", "Content_intensity_trust_total_avg", "Content_anger_total_avg", "Content_anticipation_total_avg", "Content_disgust_total_avg", "Content_fear_total_avg", "Content_joy_total_avg", "Content_negative_total_avg", "Content_positive_total_avg", "Content_sadness_total_avg", "Content_surprise_total_avg", "Content_trust_total_avg"]
emotion_df.to_csv('data_storage/final_full_sentiment.csv', index=False)
emotion_df

,Section,Title,Date,Time,Authors,URL,Content,is_spanish,Processed_Title,Processed_Content,English_Title,Translated_Content,Title_Pos,Title_Neu,Title_Neg,Title_Compound,Content_Pos,Content_Neu,Content_Neg,Content_Compound,Title_Sentiment,Content_Sentiment,Title_EmoLex_Words,Title_Total_Words,Title_anger,Title_anticipation,Title_disgust,Title_fear,Title_joy,Title_negative,Title_positive,Title_sadness,Title_surprise,Title_trust,Title_anger_intensity,Title_anticipation_intensity,Title_disgust_intensity,Title_fear_intensity,Title_joy_intensity,Title_negative_intensity,Title_positive_intensity,Title_sadness_intensity,Title_surprise_intensity,Title_trust_intensity,Title_Valence,Title_Arousal,Title_Dominance,Title_Valence_total_avg,Title_Arousal_total_avg,Title_Dominance_total_avg,Title_intensity_anger_total_avg,Title_intensity_anticipation_total_avg,Title_intensity_disgust_total_avg,Title_intensity_fear_total_avg,Title_intensity_joy_total_avg,Title_intensity_negative_total_avg,Title_intensity_positive_total_avg,Title_intensity_sadness_total_avg,Title_intensity_surprise_total_avg,Title_intensity_trust_total_avg,Title_anger_total_avg,Title_anticipation_total_avg,Title_disgust_total_avg,Title_fear_total_avg,Title_joy_total_avg,Title_negative_total_avg,Title_positive_total_avg,Title_sadness_total_avg,Title_surprise_total_avg,Title_trust_total_avg,Content_EmoLex_Words,Content_Total_Words,Content_anger,Content_anticipation,Content_disgust,Content_fear,Content_joy,Content_negative,Content_positive,Content_sadness,Content_surprise,Content_trust,Content_anger_intensity,Content_anticipation_intensity,Content_disgust_intensity,Content_fear_intensity,Content_joy_intensity,Content_negative_intensity,Content_positive_intensity,Content_sadness_intensity,Content_surprise_intensity,Content_trust_intensity,Content_Valence,Content_Arousal,Content_Dominance,Content_Valence_total_avg,Content_Arousal_total_avg,Content_Dominance_total_avg,Content_intensity_anger_total_avg,Content_intensity_anticipation_total_avg,Content_intensity_disgust_total_avg,Content_intensity_fear_total_avg,Content_intensity_joy_total_avg,Content_intensity_negative_total_avg,Content_intensity_positive_total_avg,Content_intensity_sadness_total_avg,Content_intensity_surprise_total_avg,Content_intensity_trust_total_avg,Content_anger_total_avg,Content_anticipation_total_avg,Content_disgust_total_avg,Content_fear_total_avg,Content_joy_total_avg,Content_negative_total_avg,Content_positive_total_avg,Content_sadness_total_avg,Content_surprise_total_avg,Content_trust_total_avg
0,[Sports],UCSB End Non-Conf. Portion of Season in Winnin...,2018-01-01,00:37:00,[Spencer Ault],https://dailynexus.com/2018-01-01/80339/,"After breaking through the defense, Jalen Cant...",False,ucsb end non conf portion season winning fashion,breaking defense jalen canty prepares shoot du...,None,None,0.000,0.702,0.298,0.5267,0.048,0.859,0.093,0.9755,Positive,Positive,5,8,0,1,1,0,1,0,1,1,1,1,0.000,0.664,0.0,0.000,0.000,0.0,0.0,0.000,0.0,0.266,0.661600,0.490200,0.555200,0.413500,0.306375,0.347000,0.000000,0.083000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.03325,0.000000,0.125000,0.125,0.000000,0.125000,0.000000,0.125000,0.125000,0.125000,0.125000,178,377,16,12,3,14,5,24,28,8,3,12,7.943,4.593,0.695,0.640,4.944,0.0,0.0,1.022,0.516,5.867,0.593202,0.454537,0.521504,0.380782,0.291772,0.334759,0.021069,0.012183,0.001844,0.001698,0.013114,0.0,0.0,0.002711,0.001369,0.015562,0.042440,0.031830,0.007958,0.037135,0.013263,0.063660,0.074271,0.021220,0.007958,0.031830
1,[Sports],Gauchos to Host Long Beach and Cal Poly to Ope...,2018-01-02,00:47:00,[Calvin Thrift],https://dailynexus.com/2018-01-02/gauchos-to-h...,"Sarah Porter dribbles the ball, looking for a ...",False,gaucho host long beach cal poly open big west ...,sarah porter dribble ball looking pas non conf...,None,None,0.000,0.833,0.167,0.3400,0.028,0.859,0.113,0.9952,Positive,Positive,5,10,0,1,0,0,1,0,0,0,0,0,0.000,0.406,0.0,0.000,0.500,0.0,0.0,0.000,0.0,0.000,0.702143,0.

## Keyword Analysis

In [100]:
import pandas as pd
from tqdm import tqdm
import re

# Load the data
keyword_df = pd.read_csv('data_storage/final_full_sentiment.csv')
keyword_df['Authors'] = keyword_df['Authors'].apply(clean_authors)
keyword_df['Section'] = keyword_df['Section'].apply(clean_authors)

# Define the keywords
keywords = [
    "asian", "hispanic", "latinx", "diversity", "equity", "inclusion", "equality",
    "lgbtq", "lgbtqia", "queer", "middle eastern", "black", "gay", "transgender",
    "bisexual", "lesbian", "pansexual", "asexual", "homosexual", "non-binary",
    "intersex", "aromantic", "cisgender", "coming out", "genderfluid", "privilege",
    "pronouns", "undocumented", "neurodiversity", "neurodivergent", "disability",
    "microaggression", "identity", "discrimination", "culture", "misgendering", "masculinity", "sexuality"
]

# Define the words to exclude if they follow "black" or precede "culture"
exclude_black = [
    "pepper", "hole", "coffee", "sesame", "box", "beans", "market", "bean", "holes", 
    "salt", "square", "mold", "nissan", "tea", "screens", "screen", "garlic", "shoes", 
    "bear", "ski", "puffer", "milk", "smoke", "pants", "clothing", "bicycle", "gloves", 
    "ops", "suit", "liquid", "sweatshirt", "belt", "fur", "sugar", "swan", "seeds", 
    "lipstick", "levi", "stretching", "olives", "spoiler", "guitar", "rice", "tahini", 
    "bike", "silhouette", "swimsuit", "backpacks", "page", "sedan", "corset", "skinny", 
    "dress", "mask", "mesh", "drape", "binder", "motorcycle", "rubber", "clothes", "dots", 
    "cats", "toyota", "mascara", "designer", "colors", "honda", "chevrolet", "4runner", 
    "long", "shirt", "sauce", "birds", "light", "boots", "pavement", "phone", "sand", 
    "eye", "ground", "landscape", "lace", "backpack", "charcoal", "jeans", "hoodie", 
    "bicycles", "truffle", "leather", "bmw", "denim", "attire", "jumpsuit", "raspberry", 
    "tunnel", "car", "compact", "dildo", "sweater", "sunglasses"
]

exclude_culture = ["lab", "laboratory", "cell"]

# Function to count keyword occurrences, considering exclusions
def count_keywords(row):
    if row['is_spanish']:
        title = row['English_Title'].lower() if row['English_Title'] else ""
        content = row['Translated_Content'].lower() if row['Translated_Content'] else ""
    else:
        title = row['Processed_Title'].lower() if row['Processed_Title'] else ""
        content = row['Processed_Content'].lower() if row['Processed_Content'] else ""
    
    title_count = 0
    content_count = 0
    
    # Count keywords in title
    for keyword in keywords:
        if keyword in title:
            if keyword == "black":
                # Check if excluded words follow "black"
                if any(re.search(rf"\bblack\s+{excl}\b", title) for excl in exclude_black):
                    continue
                title_count += 1
            elif keyword == "culture":
                # Check if excluded words precede "culture"
                if any(re.search(rf"\b{excl}\s+culture\b", title) for excl in exclude_culture):
                    continue
                title_count += 1
            else:
                title_count += 1

    # Count keywords in content
    for keyword in keywords:
        if keyword in content:
            if keyword == "black":
                # Check if excluded words follow "black"
                if any(re.search(rf"\bblack\s+{excl}\b", content) for excl in exclude_black):
                    continue
                content_count += 1
            elif keyword == "culture":
                # Check if excluded words precede "culture"
                if any(re.search(rf"\b{excl}\s+culture\b", content) for excl in exclude_culture):
                    continue
                content_count += 1
            else:
                content_count += 1

    return pd.Series([title_count, content_count], index=['Title_Keyword_Count', 'Content_Keyword_Count'])

# Add a progress bar to the apply function
tqdm.pandas()

# Apply the function to each row with a progress bar
keyword_df[['Title_Keyword_Count', 'Content_Keyword_Count']] = keyword_df.progress_apply(count_keywords, axis=1)

# Create Yes/No columns based on the counts
keyword_df['Title_Keyword_Flag'] = keyword_df['Title_Keyword_Count'].apply(lambda x: 'Yes' if x > 0 else 'No')
keyword_df['Content_Keyword_Flag'] = keyword_df['Content_Keyword_Count'].apply(lambda x: 'Yes' if x > 0 else 'No')

# Save the updated dataframe to a new CSV file
keyword_df.to_csv('data_storage/full_sentiment_keywords.csv', index=False)

keyword_df

100%|██████████| 6565/6565 [00:06<00:00, 1041.51it/s]


,Section,Title,Date,Time,Authors,URL,Content,is_spanish,Processed_Title,Processed_Content,English_Title,Translated_Content,Title_Pos,Title_Neu,Title_Neg,Title_Compound,Content_Pos,Content_Neu,Content_Neg,Content_Compound,Title_Sentiment,Content_Sentiment,Title_EmoLex_Words,Title_Total_Words,Title_anger,Title_anticipation,Title_disgust,Title_fear,Title_joy,Title_negative,Title_positive,Title_sadness,Title_surprise,Title_trust,Title_anger_intensity,Title_anticipation_intensity,Title_disgust_intensity,Title_fear_intensity,Title_joy_intensity,Title_negative_intensity,Title_positive_intensity,Title_sadness_intensity,Title_surprise_intensity,Title_trust_intensity,Title_Valence,Title_Arousal,Title_Dominance,Title_Valence_total_avg,Title_Arousal_total_avg,Title_Dominance_total_avg,Title_intensity_anger_total_avg,Title_intensity_anticipation_total_avg,Title_intensity_disgust_total_avg,Title_intensity_fear_total_avg,Title_intensity_joy_total_avg,Title_intensity_negative_total_avg,Title_intensity_positive_total_avg,Title_intensity_sadness_total_avg,Title_intensity_surprise_total_avg,Title_intensity_trust_total_avg,Title_anger_total_avg,Title_anticipation_total_avg,Title_disgust_total_avg,Title_fear_total_avg,Title_joy_total_avg,Title_negative_total_avg,Title_positive_total_avg,Title_sadness_total_avg,Title_surprise_total_avg,Title_trust_total_avg,Content_EmoLex_Words,Content_Total_Words,Content_anger,Content_anticipation,Content_disgust,Content_fear,Content_joy,Content_negative,Content_positive,Content_sadness,Content_surprise,Content_trust,Content_anger_intensity,Content_anticipation_intensity,Content_disgust_intensity,Content_fear_intensity,Content_joy_intensity,Content_negative_intensity,Content_positive_intensity,Content_sadness_intensity,Content_surprise_intensity,Content_trust_intensity,Content_Valence,Content_Arousal,Content_Dominance,Content_Valence_total_avg,Content_Arousal_total_avg,Content_Dominance_total_avg,Content_intensity_anger_total_avg,Content_intensity_anticipation_total_avg,Content_intensity_disgust_total_avg,Content_intensity_fear_total_avg,Content_intensity_joy_total_avg,Content_intensity_negative_total_avg,Content_intensity_positive_total_avg,Content_intensity_sadness_total_avg,Content_intensity_surprise_total_avg,Content_intensity_trust_total_avg,Content_anger_total_avg,Content_anticipation_total_avg,Content_disgust_total_avg,Content_fear_total_avg,Content_joy_total_avg,Content_negative_total_avg,Content_positive_total_avg,Content_sadness_total_avg,Content_surprise_total_avg,Content_trust_total_avg,Title_Keyword_Count,Content_Keyword_Count,Title_Keyword_Flag,Content_Keyword_Flag
0,[Sports],UCSB End Non-Conf. Portion of Season in Winnin...,2018-01-01,00:37:00,[Spencer Ault],https://dailynexus.com/2018-01-01/80339/,"After breaking through the defense, Jalen Cant...",False,ucsb end non conf portion season winning fashion,breaking defense jalen canty prepares shoot du...,None,None,0.000,0.702,0.298,0.5267,0.048,0.859,0.093,0.9755,Positive,Positive,5,8,0,1,1,0,1,0,1,1,1,1,0.000,0.664,0.0,0.000,0.000,0.0,0.0,0.000,0.0,0.266,0.661600,0.490200,0.555200,0.413500,0.306375,0.347000,0.000000,0.083000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.03325,0.000000,0.125000,0.125,0.000000,0.125000,0.000000,0.125000,0.125000,0.125000,0.125000,178,377,16,12,3,14,5,24,28,8,3,12,7.943,4.593,0.695,0.640,4.944,0.0,0.0,1.022,0.516,5.867,0.593202,0.454537,0.521504,0.380782,0.291772,0.334759,0.021069,0.012183,0.001844,0.001698,0.013114,0.0,0.0,0.002711,0.001369,0.015562,0.042440,0.031830,0.007958,0.037135,0.013263,0.063660,0.074271,0.021220,0.007958,0.031830,0,0,No,No
1,[Sports],Gauchos to Host Long Beach and Cal Poly to Ope...,2018-01-02,00:47:00,[Calvin Thrift],https://dailynexus.com/2018-01-02/gauchos-to-h...,"Sarah Porter dribbles the ball, looking for a ...",False,gaucho host long beach cal poly open big west ...,sarah porter dribble ball looking pas non conf...,None,None,0.000,0.833,0.167,0.3400,0.028,0.859,0.113,0.9952,Positive,Posit

## Chart Preparation

#### Keywords

In [103]:
final_df = pd.read_csv('data_storage/full_sentiment_keywords.csv')
final_df['Authors'] = final_df['Authors'].apply(clean_authors)
final_df['Section'] = final_df['Section'].apply(clean_authors)

final_df

,Section,Title,Date,Time,Authors,URL,Content,is_spanish,Processed_Title,Processed_Content,English_Title,Translated_Content,Title_Pos,Title_Neu,Title_Neg,Title_Compound,Content_Pos,Content_Neu,Content_Neg,Content_Compound,Title_Sentiment,Content_Sentiment,Title_EmoLex_Words,Title_Total_Words,Title_anger,Title_anticipation,Title_disgust,Title_fear,Title_joy,Title_negative,Title_positive,Title_sadness,Title_surprise,Title_trust,Title_anger_intensity,Title_anticipation_intensity,Title_disgust_intensity,Title_fear_intensity,Title_joy_intensity,Title_negative_intensity,Title_positive_intensity,Title_sadness_intensity,Title_surprise_intensity,Title_trust_intensity,Title_Valence,Title_Arousal,Title_Dominance,Title_Valence_total_avg,Title_Arousal_total_avg,Title_Dominance_total_avg,Title_intensity_anger_total_avg,Title_intensity_anticipation_total_avg,Title_intensity_disgust_total_avg,Title_intensity_fear_total_avg,Title_intensity_joy_total_avg,Title_intensity_negative_total_avg,Title_intensity_positive_total_avg,Title_intensity_sadness_total_avg,Title_intensity_surprise_total_avg,Title_intensity_trust_total_avg,Title_anger_total_avg,Title_anticipation_total_avg,Title_disgust_total_avg,Title_fear_total_avg,Title_joy_total_avg,Title_negative_total_avg,Title_positive_total_avg,Title_sadness_total_avg,Title_surprise_total_avg,Title_trust_total_avg,Content_EmoLex_Words,Content_Total_Words,Content_anger,Content_anticipation,Content_disgust,Content_fear,Content_joy,Content_negative,Content_positive,Content_sadness,Content_surprise,Content_trust,Content_anger_intensity,Content_anticipation_intensity,Content_disgust_intensity,Content_fear_intensity,Content_joy_intensity,Content_negative_intensity,Content_positive_intensity,Content_sadness_intensity,Content_surprise_intensity,Content_trust_intensity,Content_Valence,Content_Arousal,Content_Dominance,Content_Valence_total_avg,Content_Arousal_total_avg,Content_Dominance_total_avg,Content_intensity_anger_total_avg,Content_intensity_anticipation_total_avg,Content_intensity_disgust_total_avg,Content_intensity_fear_total_avg,Content_intensity_joy_total_avg,Content_intensity_negative_total_avg,Content_intensity_positive_total_avg,Content_intensity_sadness_total_avg,Content_intensity_surprise_total_avg,Content_intensity_trust_total_avg,Content_anger_total_avg,Content_anticipation_total_avg,Content_disgust_total_avg,Content_fear_total_avg,Content_joy_total_avg,Content_negative_total_avg,Content_positive_total_avg,Content_sadness_total_avg,Content_surprise_total_avg,Content_trust_total_avg,Title_Keyword_Count,Content_Keyword_Count,Title_Keyword_Flag,Content_Keyword_Flag
0,[Sports],UCSB End Non-Conf. Portion of Season in Winnin...,2018-01-01,00:37:00,[Spencer Ault],https://dailynexus.com/2018-01-01/80339/,"After breaking through the defense, Jalen Cant...",False,ucsb end non conf portion season winning fashion,breaking defense jalen canty prepares shoot du...,None,None,0.000,0.702,0.298,0.5267,0.048,0.859,0.093,0.9755,Positive,Positive,5,8,0,1,1,0,1,0,1,1,1,1,0.000,0.664,0.0,0.000,0.000,0.0,0.0,0.000,0.0,0.266,0.661600,0.490200,0.555200,0.413500,0.306375,0.347000,0.000000,0.083000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.03325,0.000000,0.125000,0.125,0.000000,0.125000,0.000000,0.125000,0.125000,0.125000,0.125000,178,377,16,12,3,14,5,24,28,8,3,12,7.943,4.593,0.695,0.640,4.944,0.0,0.0,1.022,0.516,5.867,0.593202,0.454537,0.521504,0.380782,0.291772,0.334759,0.021069,0.012183,0.001844,0.001698,0.013114,0.0,0.0,0.002711,0.001369,0.015562,0.042440,0.031830,0.007958,0.037135,0.013263,0.063660,0.074271,0.021220,0.007958,0.031830,0,0,No,No
1,[Sports],Gauchos to Host Long Beach and Cal Poly to Ope...,2018-01-02,00:47:00,[Calvin Thrift],https://dailynexus.com/2018-01-02/gauchos-to-h...,"Sarah Porter dribbles the ball, looking for a ...",False,gaucho host long beach cal poly open big west ...,sarah porter dribble ball looking pas non conf...,None,None,0.000,0.833,0.167,0.3400,0.028,0.859,0.113,0.9952,Positive,Posit

In [104]:
# Ensure the 'Date' column is in datetime format, then extract the year
final_df['Date'] = pd.to_datetime(final_df['Date'])
final_df['Year'] = final_df['Date'].dt.year

# Create a combined keyword flag
final_df['Keyword_Flag'] = final_df.apply(
    lambda row: 'Yes' if row['Title_Keyword_Flag'] == 'Yes' or row['Content_Keyword_Flag'] == 'Yes' else 'No',
    axis=1
)

# Group by year and aggregate the data
grouped_keywords_df = final_df.groupby('Year').agg(
    Article_Count=('URL', 'count'),
    Title_Keyword_Count=('Title_Keyword_Count', 'sum'),
    Content_Keyword_Count=('Content_Keyword_Count', 'sum'),
    Title_Flag_Yes_Count=('Title_Keyword_Flag', lambda x: (x == 'Yes').sum()),
    Title_Flag_No_Count=('Title_Keyword_Flag', lambda x: (x == 'No').sum()),
    Content_Flag_Yes_Count=('Content_Keyword_Flag', lambda x: (x == 'Yes').sum()),
    Content_Flag_No_Count=('Content_Keyword_Flag', lambda x: (x == 'No').sum()),
    Keyword_Flag_Yes_Count=('Keyword_Flag', lambda x: (x == 'Yes').sum()),
    Keyword_Flag_No_Count=('Keyword_Flag', lambda x: (x == 'No').sum())
).reset_index()

# Calculate total keyword count and average keywords per article
grouped_keywords_df['Total_Keyword_Count'] = grouped_keywords_df['Title_Keyword_Count'] + grouped_keywords_df['Content_Keyword_Count']
grouped_keywords_df['Average_Keyword_Per_Article'] = grouped_keywords_df['Total_Keyword_Count'] / grouped_keywords_df['Article_Count']

# Proportion of articles with a keyword overall
grouped_keywords_df['Keyword_Flag_Proportion'] = grouped_keywords_df['Keyword_Flag_Yes_Count'] / grouped_keywords_df['Article_Count']

# Do title proportion
grouped_keywords_df['Title_Flag_Proportion'] = grouped_keywords_df['Title_Flag_Yes_Count'] / grouped_keywords_df['Article_Count']

# Save keyword proportion to csv file
grouped_keywords_df[['Year', 'Keyword_Flag_Proportion']].to_csv('chart_storage/proportion_of_articles_with_keywords_by_year.csv', index=False)

# Display the resulting dataframe
grouped_keywords_df


,Year,Article_Count,Title_Keyword_Count,Content_Keyword_Count,Title_Flag_Yes_Count,Title_Flag_No_Count,Content_Flag_Yes_Count,Content_Flag_No_Count,Keyword_Flag_Yes_Count,Keyword_Flag_No_Count,Total_Keyword_Count,Average_Keyword_Per_Article,Keyword_Flag_Proportion,Title_Flag_Proportion
0,2018,1157,22,552,21,1136,332,825,333,824,574,0.496111,0.287813,0.018150
1,2019,1295,46,771,41,1254,418,877,420,875,817,0.630888,0.324324,0.031660
2,2020,1033,34,611,30,1003,330,703,330,703,645,0.624395,0.319458,0.029042
3,2021,875,38,683,32,843,303,572,305,570,721,0.824000,0.348571,0.036571
4,2022,920,50,823,40,880,360,560,360,560,873,0.948913,0.391304,0.043478
5,2023,787,52,867,47,740,344,443,344,443,919,1.167726,0.437103,0.059720
6,2024,498,33,514,30,468,249,249,250,248,547,1.098394,0.502008,0.060241


In [293]:
final_df = pd.read_csv('data_storage/full_sentiment_keywords.csv')

# Ensure the 'Date' column is in datetime format, then extract the year and month
final_df['Date'] = pd.to_datetime(final_df['Date'])
final_df['Year'] = final_df['Date'].dt.year
final_df['Month'] = final_df['Date'].dt.month

# Create a combined keyword flag
final_df['Keyword_Flag'] = final_df.apply(
    lambda row: 'Yes' if row['Title_Keyword_Flag'] == 'Yes' or row['Content_Keyword_Flag'] == 'Yes' else 'No',
    axis=1
)

# Group by year and month and aggregate the data
grouped_keywords_df = final_df.groupby(['Year', 'Month']).agg(
    Article_Count=('URL', 'count'),
    Title_Keyword_Count=('Title_Keyword_Count', 'sum'),
    Content_Keyword_Count=('Content_Keyword_Count', 'sum'),
    Title_Flag_Yes_Count=('Title_Keyword_Flag', lambda x: (x == 'Yes').sum()),
    Title_Flag_No_Count=('Title_Keyword_Flag', lambda x: (x == 'No').sum()),
    Content_Flag_Yes_Count=('Content_Keyword_Flag', lambda x: (x == 'Yes').sum()),
    Content_Flag_No_Count=('Content_Keyword_Flag', lambda x: (x == 'No').sum()),
    Keyword_Flag_Yes_Count=('Keyword_Flag', lambda x: (x == 'Yes').sum()),
    Keyword_Flag_No_Count=('Keyword_Flag', lambda x: (x == 'No').sum())
).reset_index()

# Calculate total keyword count and average keywords per article
grouped_keywords_df['Total_Keyword_Count'] = grouped_keywords_df['Title_Keyword_Count'] + grouped_keywords_df['Content_Keyword_Count']
grouped_keywords_df['Average_Keyword_Per_Article'] = grouped_keywords_df['Total_Keyword_Count'] / grouped_keywords_df['Article_Count']

# Proportion of articles with a keyword overall
grouped_keywords_df['Keyword_Flag_Proportion'] = grouped_keywords_df['Keyword_Flag_Yes_Count'] / grouped_keywords_df['Article_Count']

# Proportion of articles with keywords in title
grouped_keywords_df['Title_Flag_Proportion'] = grouped_keywords_df['Title_Flag_Yes_Count'] / grouped_keywords_df['Article_Count']

# Save keyword proportion to CSV file by month
grouped_keywords_df[['Year', 'Month', 'Keyword_Flag_Proportion']].to_csv('chart_storage/proportion_of_articles_with_keywords_by_month.csv', index=False)

# Filter
grouped_keywords_df = grouped_keywords_df[['Year', 'Month', 'Keyword_Flag_Proportion']][(grouped_keywords_df['Year'] >= 2022)]

# Drop last row
grouped_keywords_df = grouped_keywords_df[:-1]

# Save keyword proportion to CSV file by month
grouped_keywords_df.to_csv('chart_storage/modern_proportion_of_articles_with_keywords_by_month.csv', index=False)

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 120)

# Display the resulting dataframe
grouped_keywords_df

,Year,Month,Keyword_Flag_Proportion
0,2018,1,0.192771
1,2018,2,0.350000
2,2018,3,0.245614
3,2018,4,0.281046
4,2018,5,0.277778
5,2018,6,0.152174
6,2018,7,0.240000
7,2018,8,0.476190
8,2018,9,0.354167
9,2018,10,0.248227


In [123]:
keywords_month = pd.read_csv('chart_storage/proportion_of_articles_with_keywords_by_month.csv')
keywords_month = keywords_month.groupby('Month').agg({'Keyword_Flag_Proportion': 'mean'}).reset_index()

# Save to a new CSV file
keywords_month.to_csv('chart_storage/average_keyword_proportion_by_month.csv', index=False)

In [5]:
inal_df = pd.read_csv('data_storage/full_sentiment_keywords.csv')

# Ensure the 'Date' column is in datetime format, then extract the year and month
final_df['Date'] = pd.to_datetime(final_df['Date'])
final_df['Year'] = final_df['Date'].dt.year
final_df['Month'] = final_df['Date'].dt.month

# Define the keywords
keywords = [
    "asian", "hispanic", "latinx", "diversity", "equity", "inclusion", "equality",
    "lgbtq", "lgbtqia", "queer", "middle eastern", "black", "gay", "transgender",
    "bisexual", "lesbian", "pansexual", "asexual", "homosexual", "non-binary",
    "intersex", "aromantic", "cisgender", "coming out", "genderfluid", "privilege",
    "pronouns", "undocumented", "neurodiversity", "neurodivergent", "disability",
    "microaggression", "identity", "discrimination", "culture", "misgendering", "masculinity", "sexuality"
]

# Function to find keywords in the content
def find_keywords(content, keywords):
    found_keywords = [kw for kw in keywords if kw in content]
    return found_keywords

# Apply the function to the content column
final_df['Content_Keywords_List'] = final_df['Content'].apply(lambda x: find_keywords(x, keywords) if pd.notnull(x) else [])

# Display rows where the Content_Keyword_Flag is 'Yes'
final_df_with_keywords = final_df[final_df['Content_Keyword_Flag'] == 'Yes'][['Content', 'Content_Keywords_List', 'Year', 'Month']]

# Use pd series to get content keywords list stack
final_df_with_keywords = final_df_with_keywords.set_index('Content').apply(lambda x: x.apply(pd.Series).stack()).reset_index()

# filter month as 6 and year as 2024
final_df_with_keywords = final_df_with_keywords[(final_df_with_keywords['Year'] == 2024) & (final_df_with_keywords['Month'] == 6)]

final_df_with_keywords['Content_Keywords_List'].value_counts()

C:\Users\yeahs\AppData\Local\Temp\ipykernel_23120\2931386740.py:30: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  final_df_with_keywords = final_df_with_keywords.set_index('Content').apply(lambda x: x.apply(pd.Series).stack()).reset_index()


culture        2
identity       2
diversity      1
masculinity    1
black          1
queer          1
Name: Content_Keywords_List, dtype: int64

In [8]:
final_df[final_df['Content_Keyword_Flag'] == 'Yes'][['Content', 'Content_Keywords_List', 'Year', 'Month']][final_df['Year'] == 2024][final_df['Month'] == 6]

C:\Users\yeahs\AppData\Local\Temp\ipykernel_23120\2672067775.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  final_df[final_df['Content_Keyword_Flag'] == 'Yes'][['Content', 'Content_Keywords_List', 'Year', 'Month']][final_df['Year'] == 2024][final_df['Month'] == 6]


,Content,Content_Keywords_List,Year,Month
6533,The Associated Students Investment Advisory Co...,[],2024,6
6537,"Dearest Nexus Reader, My name is Pricila Flor...","[black, privilege]",2024,6
6538,"To kick off the beginning of Pride Month, the ...",[queer],2024,6
6540,"One UC Santa Barbara student, clad in a silver...",[culture],2024,6
6541,This article was edited on June 7 to correct t...,[],2024,6
6543,"Courtesy of Universal Pictures ""The Fall Guy,""...",[masculinity],2024,6
6544,Courtesy of Ed Miller/Netflix Like many fellow...,[],2024,6
6546,Courtesy of Fay Harvey and @wimucsb on Instagr...,"[diversity, identity, culture]",2024,6
6547,The UC Santa Barbara Associated Students 75th ...,[identity],2024,6
6548,CORRECTION [5/23/2024 2:30 p.m.]: A previous v...,[culture],2024,6


In [4]:
final_df

,Section,Title,Date,Time,Authors,URL,Content,is_spanish,Processed_Title,Processed_Content,...,Content_sadness_total_avg,Content_surprise_total_avg,Content_trust_total_avg,Title_Keyword_Count,Content_Keyword_Count,Title_Keyword_Flag,Content_Keyword_Flag,Year,Month,Content_Keywords_List
0,['Sports'],UCSB End Non-Conf. Portion of Season in Winnin...,2018-01-01,00:37:00,['Spencer Ault'],https://dailynexus.com/2018-01-01/80339/,"After breaking through the defense, Jalen Cant...",False,ucsb end non conf portion season winning fashion,breaking defense jalen canty prepares shoot du...,...,0.021220,0.007958,0.031830,0,0,No,No,2018,1,[]
1,['Sports'],Gauchos to Host Long Beach and Cal Poly to Ope...,2018-01-02,00:47:00,['Calvin Thrift'],https://dailynexus.com/2018-01-02/gauchos-to-h...,"Sarah Porter dribbles the ball, looking for a ...",False,gaucho host long beach cal poly open big west ...,sarah porter dribble ball looking pas non conf...,...,0.023599,0.020649,0.061947,0,0,No,No,2018,1,[]
2,['Sports'],Gauchos Outlast 49ers in Conference Opener,2018-01-05,19:30:00,['Calvin Thrift'],https://dailynexus.com/2018-01-05/gauchos-outl...,Sarah Porter brings the ball up the court. Des...,False,gaucho outlast conference opener,sarah porter brings ball court despite another...,...,0.031046,0.024510,0.044118,0,1,No,Yes,2018,1,[identity]
3,['News'],Earthquake Measuring 3.8 Reported Near Isla Vi...,2018-01-06,19:49:00,['Hannah Jackson'],https://dailynexus.com/2018-01-06/earthquake-m...,A 3.8 magnitude earthquake was reported off th...,False,earthquake measuring reported near isla vista ...,magnitude earthquake reported coast isla vista...,...,0.079208,0.079208,0.049505,0,0,No,No,2018,1,[]
4,['Artsweek'],The Friday Review: Saturation III by Brockhampton,2018-01-06,15:14:00,['Konrad Neithercutt'],https://dailynexus.com/2018-01-06/the-friday-r...,The Internet's favorite boy band just solidifi...,False,friday review saturation iii brockhampton,internet favorite boy band solidified hip hop ...,...,0.023095,0.018476,0.043880,0,1,No,Yes,2018,1,[diversity]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6560,['Daily Stench'],Help! The girls on the high school tours scare me,2024-07-14,04:20:00,['Joseph R. Biden'],https://dailynexus.com/2024-07-14/help-the-gir...,It was a classic spring quarter Sunday: I woke...,False,help girl high school tour scare,classic spring quarter sunday woke random bed ...,...,0.037671,0.010274,0.041096,0,0,No,No,2024,7,[]
6561,['Artsweek'],Cage the Elephant at the Santa Barbara Bowl: A...,2024-07-16,11:53:00,['Lauren Chiou'],https://dailynexus.com/2024-07-16/cage-the-ele...,American rock band Cage the Elephant performed...,False,cage elephant santa barbara bowl electrifying ...,american rock band cage elephant performed san...,...,0.031637,0.023384,0.055021,0,0,No,No,2024,7,[]
6562,['Artsweek'],"""Mustard Man"": The ""post coming-of-age-story""...",2024-07-16,16:05:00,['Kyra Schimpf'],https://dailynexus.com/2024-07-16/mustard-man-...,"Courtesy of ""Mustard Man"" crew Fresh off its C...",False,mustard man post coming age story music mayhem...,courtesy mustard man crew fresh california pre...,...,0.025947,0.014727,0.060309,0,0,No,No,2024,7,[black]
6563,['On The Menu'],Strawberry Avocado Toast: A Summer Delight,2024-07-18,10:00:00,['Pihu Jain'],https://dailynexus.com/2024-07-18/117146/,"A perfect blend of the sweet and savory, this ...",False,strawberry avocado toast summer delight,perfect blend sweet savory recipe sure delight...,...,0.012195,0.042683,0.097561,0,0,No,No,2024,7,[]


In [110]:
# Define the keywords
keywords = [
    "asian", "hispanic", "latinx", "diversity", "equity", "inclusion", "equality",
    "lgbtq", "lgbtqia", "queer", "middle eastern", "black", "gay", "transgender",
    "bisexual", "lesbian", "pansexual", "asexual", "homosexual", "non-binary",
    "intersex", "aromantic", "cisgender", "coming out", "genderfluid", "privilege",
    "pronouns", "undocumented", "neurodiversity", "neurodivergent", "disability",
    "microaggression", "identity", "discrimination", "culture", "misgendering", "masculinity", "sexuality"
]

# Function to find all words preceding "diversity" in the content
def find_words_before_diversity(content):
    matches = re.findall(r'\b(\w+)\b(?=\s+identity\b)', content.lower())
    return matches

# Apply the function to the content column
final_df['Words_Before_Diversity'] = final_df['Content'].apply(lambda x: find_words_before_diversity(x) if pd.notnull(x) else [])

# Filter out rows where there are words before "diversity"
filtered_df = final_df[final_df['Words_Before_Diversity'].apply(len) > 0]

# Explode the 'Words_Before_Diversity' column to have one entry per row
exploded_df = filtered_df.explode('Words_Before_Diversity')

# Count occurrences of each word preceding "diversity"
words_before_diversity_counts = exploded_df['Words_Before_Diversity'].value_counts().reset_index()
words_before_diversity_counts.columns = ['Word', 'Count']

words_before_diversity_counts['']

,Word,Count
0,my,64
1,their,51
2,of,25
3,gender,22
4,s,21
...,...,...
98,distinct,1
99,binational,1
100,christian,1
101,theorized,1


#### Title Sentiment

In [111]:
# Load the data
final_df = pd.read_csv('data_storage/full_sentiment_keywords.csv')
final_df['Authors'] = final_df['Authors'].apply(clean_authors)
final_df['Section'] = final_df['Section'].apply(clean_authors)
final_df['Year'] = final_df['Date'].str[:4].astype(int)

# Group title sentiment by year
grouped_title_sentiment_df = final_df.groupby('Year').agg(
    Article_Count=('URL', 'count'),
    Title_Pos_Count=('Title_Sentiment', lambda x: (x == 'Positive').sum()),
    Title_Neu_Count=('Title_Sentiment', lambda x: (x == 'Neutral').sum()),
    Title_Neg_Count=('Title_Sentiment', lambda x: (x == 'Negative').sum())
).reset_index() 

# Calculate proportions
grouped_title_sentiment_df['Title_Pos_Proportion'] = grouped_title_sentiment_df['Title_Pos_Count'] / grouped_title_sentiment_df['Article_Count']
grouped_title_sentiment_df['Title_Neu_Proportion'] = grouped_title_sentiment_df['Title_Neu_Count'] / grouped_title_sentiment_df['Article_Count']
grouped_title_sentiment_df['Title_Neg_Proportion'] = grouped_title_sentiment_df['Title_Neg_Count'] / grouped_title_sentiment_df['Article_Count']

# Drop all keywords except Year and Proportion
grouped_title_sentiment_df = grouped_title_sentiment_df[['Year', 'Title_Pos_Proportion', 'Title_Neu_Proportion', 'Title_Neg_Proportion']]

# Save to a new CSV file
grouped_title_sentiment_df.to_csv('chart_storage/all_title_overall_sentiment_by_year.csv', index=False)

grouped_title_sentiment_df

,Year,Title_Pos_Proportion,Title_Neu_Proportion,Title_Neg_Proportion
0,2018,0.261020,0.515125,0.223855
1,2019,0.311197,0.492664,0.196139
2,2020,0.293320,0.506292,0.200387
3,2021,0.269714,0.522286,0.208000
4,2022,0.313043,0.518478,0.168478
5,2023,0.315121,0.526048,0.158831
6,2024,0.271084,0.556225,0.172691


In [112]:
# Load the data
final_df = pd.read_csv('data_storage/full_sentiment_keywords.csv')
final_df['Authors'] = final_df['Authors'].apply(clean_authors)
final_df['Section'] = final_df['Section'].apply(clean_authors)
final_df['Year'] = final_df['Date'].str[:4].astype(int)

final_title_yes_df = final_df[final_df['Title_Keyword_Flag'] == 'Yes']

# Group title sentiment by year
grouped_title_sentiment_df = final_title_yes_df.groupby('Year').agg(
    Article_Count=('URL', 'count'),
    Title_Pos_Count=('Title_Sentiment', lambda x: (x == 'Positive').sum()),
    Title_Neu_Count=('Title_Sentiment', lambda x: (x == 'Neutral').sum()),
    Title_Neg_Count=('Title_Sentiment', lambda x: (x == 'Negative').sum())
).reset_index() 

# Calculate proportions
grouped_title_sentiment_df['Title_Pos_Proportion'] = grouped_title_sentiment_df['Title_Pos_Count'] / grouped_title_sentiment_df['Article_Count']
grouped_title_sentiment_df['Title_Neu_Proportion'] = grouped_title_sentiment_df['Title_Neu_Count'] / grouped_title_sentiment_df['Article_Count']
grouped_title_sentiment_df['Title_Neg_Proportion'] = grouped_title_sentiment_df['Title_Neg_Count'] / grouped_title_sentiment_df['Article_Count']

# Drop all keywords except Year and Proportion
grouped_title_sentiment_df = grouped_title_sentiment_df[['Year', 'Title_Pos_Proportion', 'Title_Neu_Proportion', 'Title_Neg_Proportion']]

# Save to a new CSV file
grouped_title_sentiment_df.to_csv('chart_storage/dei_title_overall_sentiment_by_year.csv', index=False)

grouped_title_sentiment_df

,Year,Title_Pos_Proportion,Title_Neu_Proportion,Title_Neg_Proportion
0,2018,0.380952,0.333333,0.285714
1,2019,0.292683,0.560976,0.146341
2,2020,0.433333,0.333333,0.233333
3,2021,0.218750,0.656250,0.125000
4,2022,0.375000,0.625000,0.000000
5,2023,0.276596,0.574468,0.148936
6,2024,0.433333,0.433333,0.133333


#### Content Sentiment


In [169]:
# Load the data
final_df = pd.read_csv('data_storage/full_sentiment_keywords.csv')
final_df['Authors'] = final_df['Authors'].apply(clean_authors)
final_df['Section'] = final_df['Section'].apply(clean_authors)
final_df['Year'] = final_df['Date'].str[:4].astype(int)
final_df['Month'] = final_df['Date'].str[5:7].astype(int)

# Number of total emotion words as a dataframe for each emotion by year
emotion_words_df = final_df.groupby(['Year', 'Month']).agg(
    title_anger=('Title_anger', 'sum'),
    title_anticipation=('Title_anticipation', 'sum'),
    title_disgust=('Title_disgust', 'sum'),
    title_fear=('Title_fear', 'sum'),
    title_joy=('Title_joy', 'sum'),
    title_sadness=('Title_sadness', 'sum'),
    title_surprise=('Title_surprise', 'sum'),
    title_trust=('Title_trust', 'sum'),
    title_words=('Title_Total_Words', 'sum'),
    content_anger=('Content_anger', 'sum'),
    content_anticipation=('Content_anticipation', 'sum'),
    content_disgust=('Content_disgust', 'sum'),
    content_fear=('Content_fear', 'sum'),
    content_joy=('Content_joy', 'sum'),
    content_sadness=('Content_sadness', 'sum'),
    content_surprise=('Content_surprise', 'sum'),
    content_trust=('Content_trust', 'sum'),
    content_words=('Content_Total_Words', 'sum')
).reset_index()

# Group emotion words by year
emotion_words_df_year = emotion_words_df.groupby('Year').sum().drop('Month', axis=1).reset_index()
emotion_words_df_year

,Year,title_anger,title_anticipation,title_disgust,title_fear,title_joy,title_sadness,title_surprise,title_trust,title_words,content_anger,content_anticipation,content_disgust,content_fear,content_joy,content_sadness,content_surprise,content_trust,content_words
0,2018,297,411,120,308,296,259,158,485,7250,11658,22403,5903,13976,15659,11407,8601,29112,416957
1,2019,335,438,126,347,358,273,191,612,8719,13578,25972,6898,16463,18701,13002,9480,35392,501779
2,2020,221,402,105,321,294,253,170,576,7480,11034,22381,5823,14974,14565,11694,8073,30297,421623
3,2021,201,364,80,278,256,228,154,471,6154,9135,19355,4797,12565,13312,10272,7053,25133,348834
4,2022,189,367,87,236,279,189,133,461,6300,9285,22173,4690,11697,15768,10223,7972,28674,388399
5,2023,131,314,72,151,232,151,107,379,5453,7262,17908,4004,9249,13099,7826,6356,23491,335439
6,2024,76,224,43,105,154,85,85,251,3402,5064,12555,2675,6508,9499,5496,4838,17079,232023


In [242]:
# Load data
final_df = pd.read_csv('data_storage/full_sentiment_keywords.csv')
final_df['Authors'] = final_df['Authors'].apply(clean_authors)
final_df['Section'] = final_df['Section'].apply(clean_authors)
final_df['Year'] = final_df['Date'].str[:4].astype(int)
final_df['Month'] = final_df['Date'].str[5:7].astype(int)

# Define the relevant emotion columns for title and content
title_columns = [
    'Title_anger', 'Title_anticipation', 'Title_disgust', 'Title_fear', 
    'Title_joy', 'Title_sadness', 
    'Title_surprise', 'Title_trust'
]

content_columns = [
    'Content_anger', 'Content_anticipation', 'Content_disgust', 'Content_fear', 
    'Content_joy', 'Content_sadness', 
    'Content_surprise', 'Content_trust'
]

# Calculate proportions and create new columns in a new DataFrame
proportions_df = final_df.copy()
for col in title_columns:
    proportions_df[col + '_Proportion'] = proportions_df[col] / proportions_df['Title_Total_Words']

for col in content_columns:
    proportions_df[col + '_Proportion'] = proportions_df[col] / proportions_df['Content_Total_Words']

# Content Emotion Proportions
prop_content = proportions_df[['Year'] + content_columns + ['Content_Total_Words'] + [col + '_Proportion' for col in content_columns]].groupby('Year').mean()
prop_content = prop_content.drop(columns=['Content_anger', 'Content_anticipation', 'Content_disgust', 'Content_fear', 'Content_joy', 'Content_sadness', 'Content_surprise', 'Content_trust', 'Content_Total_Words'])
prop_content.reset_index(inplace=True)

# Title Emotion Proportions
prop_title = proportions_df[['Year'] + title_columns + ['Title_Total_Words'] + [col + '_Proportion' for col in title_columns]].groupby('Year').mean()
prop_title = prop_title.drop(columns=['Title_anger', 'Title_anticipation', 'Title_disgust', 'Title_fear', 'Title_joy', 'Title_sadness', 'Title_surprise', 'Title_trust', 'Title_Total_Words'])
prop_title.reset_index(inplace=True)

# Concatenate the two dataframes and drop the year duplicate
prop_emotion = pd.concat([prop_title, prop_content], axis=1)
prop_emotion = prop_emotion.loc[:, ~prop_emotion.columns.duplicated()]
prop_emotion

# Save the updated dataframe to a new CSV file
prop_emotion.to_csv('chart_storage/avg_emotion_proportion_by_year.csv', index=False)

# Display the resulting dataframe
prop_emotion

,Year,Title_anger_Proportion,Title_anticipation_Proportion,Title_disgust_Proportion,Title_fear_Proportion,Title_joy_Proportion,Title_sadness_Proportion,Title_surprise_Proportion,Title_trust_Proportion,Content_anger_Proportion,Content_anticipation_Proportion,Content_disgust_Proportion,Content_fear_Proportion,Content_joy_Proportion,Content_sadness_Proportion,Content_surprise_Proportion,Content_trust_Proportion
0,2018,0.039658,0.057642,0.017753,0.041784,0.041176,0.035881,0.021912,0.065020,0.027620,0.053758,0.014049,0.033397,0.037384,0.027008,0.020785,0.069667
1,2019,0.038575,0.051574,0.015644,0.038889,0.043861,0.032541,0.022502,0.070608,0.026949,0.051937,0.013958,0.032496,0.037713,0.025703,0.019431,0.069333
2,2020,0.029552,0.056014,0.014309,0.042173,0.043563,0.033305,0.022599,0.074491,0.024505,0.053846,0.013220,0.033933,0.034918,0.026572,0.018614,0.069867
3,2021,0.032507,0.059335,0.014464,0.045880,0.041089,0.037636,0.025314,0.072902,0.024643,0.055784,0.012901,0.034180,0.038328,0.027437,0.019575,0.070401
4,2022,0.028431,0.058949,0.014160,0.035538,0.045188,0.029886,0.020062,0.070495,0.023759,0.058610,0.012255,0.030714,0.040806,0.025793,0.020537,0.073348
5,2023,0.021923,0.058818,0.012331,0.025055,0.040639,0.025095,0.018617,0.064901,0.021761,0.052602,0.011977,0.027905,0.038115,0.023560,0.018780,0.067457
6,2024,0.021369,0.071051,0.012787,0.030619,0.045311,0.024589,0.023988,0.069950,0.021861,0.052250,0.011925,0.028311,0.039273,0.022862,0.020008,0.070416


In [245]:
# Load data
final_df = pd.read_csv('data_storage/full_sentiment_keywords.csv')
final_df['Authors'] = final_df['Authors'].apply(clean_authors)
final_df['Section'] = final_df['Section'].apply(clean_authors)
final_df['Year'] = final_df['Date'].str[:4].astype(int)
final_df['Month'] = final_df['Date'].str[5:7].astype(int)

# Define the relevant emotion columns for title and content
title_columns = [
    'Title_anger', 'Title_anticipation', 'Title_disgust', 'Title_fear', 
    'Title_joy', 'Title_sadness', 
    'Title_surprise', 'Title_trust'
]

content_columns = [
    'Content_anger', 'Content_anticipation', 'Content_disgust', 'Content_fear', 
    'Content_joy', 'Content_sadness', 
    'Content_surprise', 'Content_trust'
]

final_df['Keyword_Flag'] = final_df.apply(
    lambda row: 'Yes' if row['Title_Keyword_Flag'] == 'Yes' or row['Content_Keyword_Flag'] == 'Yes' else 'No',
    axis=1
)

# Calculate proportions and create new columns in a new DataFrame
proportions_df = final_df[final_df['Keyword_Flag'] == 'Yes'].copy()
for col in title_columns:
    proportions_df[col + '_Proportion'] = proportions_df[col] / proportions_df['Title_Total_Words']

for col in content_columns:
    proportions_df[col + '_Proportion'] = proportions_df[col] / proportions_df['Content_Total_Words']

# Content Emotion Proportions
prop_content = proportions_df[['Year'] + content_columns + ['Content_Total_Words'] + [col + '_Proportion' for col in content_columns]].groupby('Year').mean()
prop_content = prop_content.drop(columns=['Content_anger', 'Content_anticipation', 'Content_disgust', 'Content_fear', 'Content_joy', 'Content_sadness', 'Content_surprise', 'Content_trust', 'Content_Total_Words'])
prop_content.reset_index(inplace=True)

# Title Emotion Proportions
prop_title = proportions_df[['Year'] + title_columns + ['Title_Total_Words'] + [col + '_Proportion' for col in title_columns]].groupby('Year').mean()
prop_title = prop_title.drop(columns=['Title_anger', 'Title_anticipation', 'Title_disgust', 'Title_fear', 'Title_joy', 'Title_sadness', 'Title_surprise', 'Title_trust', 'Title_Total_Words'])
prop_title.reset_index(inplace=True)

# Concatenate the two dataframes and drop the year duplicate
prop_emotion = pd.concat([prop_title, prop_content], axis=1)
prop_emotion = prop_emotion.loc[:, ~prop_emotion.columns.duplicated()]
prop_emotion

# Save the updated dataframe to a new CSV file
prop_emotion.to_csv('chart_storage/dei_avg_emotion_proportion_by_year.csv', index=False)

# Display the resulting dataframe
prop_emotion

,Year,Title_anger_Proportion,Title_anticipation_Proportion,Title_disgust_Proportion,Title_fear_Proportion,Title_joy_Proportion,Title_sadness_Proportion,Title_surprise_Proportion,Title_trust_Proportion,Content_anger_Proportion,Content_anticipation_Proportion,Content_disgust_Proportion,Content_fear_Proportion,Content_joy_Proportion,Content_sadness_Proportion,Content_surprise_Proportion,Content_trust_Proportion
0,2018,0.039893,0.054816,0.015673,0.041320,0.040015,0.029440,0.022536,0.071423,0.027646,0.052907,0.014572,0.033724,0.039675,0.025759,0.020205,0.072210
1,2019,0.037979,0.043379,0.014129,0.038714,0.044641,0.030003,0.020163,0.080679,0.025548,0.051047,0.013601,0.031408,0.039525,0.025190,0.018136,0.071898
2,2020,0.031284,0.052608,0.011315,0.038110,0.039157,0.031677,0.018416,0.087516,0.026447,0.051855,0.013890,0.035420,0.035305,0.027019,0.019029,0.076109
3,2021,0.028903,0.060568,0.014143,0.046166,0.046024,0.037503,0.030653,0.075988,0.025517,0.055585,0.013688,0.035132,0.041778,0.027591,0.021156,0.073514
4,2022,0.025168,0.052454,0.011574,0.032052,0.045804,0.028294,0.019279,0.071509,0.021874,0.055748,0.012039,0.029862,0.041371,0.024854,0.019489,0.075686
5,2023,0.014813,0.050715,0.009994,0.019311,0.039165,0.018379,0.017670,0.074537,0.020600,0.053266,0.011666,0.026226,0.039786,0.022758,0.018261,0.073558
6,2024,0.013816,0.063360,0.011903,0.025326,0.049858,0.019899,0.024975,0.081815,0.021604,0.052523,0.011768,0.027338,0.041299,0.022477,0.020302,0.076508


#### Extra Analysis

In [273]:
# Load data
final_df = pd.read_csv('data_storage/full_sentiment_keywords.csv')
final_df['Authors'] = final_df['Authors'].apply(clean_authors)
final_df['Section'] = final_df['Section'].apply(clean_authors)
final_df['Year'] = final_df['Date'].str[:4].astype(int)


# Pull title, title pos, title neu, title neg, title compound, title sentiment
title_sentiment_df = final_df[['Year', 'Title', 'Title_Pos', 'Title_Neu', 'Title_Neg', 'Title_Compound', 'Title_Sentiment']]


for title in title_sentiment_df[title_sentiment_df['Title_Sentiment'] == "Negative"].head(15)['Title']:
    print(title)

title_sentiment_df[title_sentiment_df['Title_Sentiment'] == "Negative"].head(15)

Gauchos Suffer First Conf. Loss at UC Irvine
Santa Barbara Community Mourns Lives Lost in Montecito Mudslides
I.V. Homeless Population Weathers Fire and Rain
Local Businesses Cope With Food Shortages as Highway 101 Remains Closed
Sexual Assault Reported in UCSB Student Housing
Argument in the Office: New Year New You? Or More of the Same?
Study Abroad Student Misses Memo, Shows Up 1 Week Early
Sad! Thousands of New Year's Resolutions Left Abandoned on Streets of DP
Fuck It, I'm Rating the Birds
Affected by Disaster, Professors Took Action
Men's Volleyball Remains on the Road for Match Against Loyola (Ill.)
Students Still Strained by Travel Ban 1 Year Later
Gauchos Struggle to Knock Off Unbeaten UC Davis on the Road
Composer Souren Baronian Dazes MCC Armenian Jazz
No. 14 Gauchos Drop Second-Straight While on the Road at Lewis


,Year,Title,Title_Pos,Title_Neu,Title_Neg,Title_Compound,Title_Sentiment
14,2018,Gauchos Suffer First Conf. Loss at UC Irvine,0.492,0.508,0.0,-0.7003,Negative
15,2018,Santa Barbara Community Mourns Lives Lost in M...,0.449,0.551,0.0,-0.6908,Negative
20,2018,I.V. Homeless Population Weathers Fire and Rain,0.286,0.714,0.0,-0.3400,Negative
22,2018,Local Businesses Cope With Food Shortages as H...,0.138,0.862,0.0,-0.1531,Negative
24,2018,Sexual Assault Reported in UCSB Student Housing,0.388,0.612,0.0,-0.5859,Negative
29,2018,Argument in the Office: New Year New You? Or M...,0.192,0.808,0.0,-0.4329,Negative
30,2018,"Study Abroad Student Misses Memo, Shows Up 1 W...",0.174,0.826,0.0,-0.2263,Negative
36,2018,Sad! Thousands of New Year's Resolutions Left ...,0.390,0.610,0.0,-0.7500,Negative
39,2018,"Fuck It, I'm Rating the Birds",0.412,0.588,0.0,-0.5423,Negative
43,2018,"Affected by Disaster, Professors Took Action",0.588,0.412,0.0,-0.6908,Negative


In [251]:
final_df

,Section,Title,Date,Time,Authors,URL,Content,is_spanish,Processed_Title,Processed_Content,English_Title,Translated_Content,Title_Pos,Title_Neu,Title_Neg,Title_Compound,Content_Pos,Content_Neu,Content_Neg,Content_Compound,Title_Sentiment,Content_Sentiment,Title_EmoLex_Words,Title_Total_Words,Title_anger,Title_anticipation,Title_disgust,Title_fear,Title_joy,Title_negative,Title_positive,Title_sadness,Title_surprise,Title_trust,Title_anger_intensity,Title_anticipation_intensity,Title_disgust_intensity,Title_fear_intensity,Title_joy_intensity,Title_negative_intensity,Title_positive_intensity,Title_sadness_intensity,Title_surprise_intensity,Title_trust_intensity,Title_Valence,Title_Arousal,Title_Dominance,Title_Valence_total_avg,Title_Arousal_total_avg,Title_Dominance_total_avg,Title_intensity_anger_total_avg,Title_intensity_anticipation_total_avg,Title_intensity_disgust_total_avg,Title_intensity_fear_total_avg,Title_intensity_joy_total_avg,Title_intensity_negative_total_avg,Title_intensity_positive_total_avg,Title_intensity_sadness_total_avg,Title_intensity_surprise_total_avg,Title_intensity_trust_total_avg,Title_anger_total_avg,Title_anticipation_total_avg,Title_disgust_total_avg,Title_fear_total_avg,Title_joy_total_avg,Title_negative_total_avg,Title_positive_total_avg,Title_sadness_total_avg,Title_surprise_total_avg,Title_trust_total_avg,Content_EmoLex_Words,Content_Total_Words,Content_anger,Content_anticipation,Content_disgust,Content_fear,Content_joy,Content_negative,Content_positive,Content_sadness,Content_surprise,Content_trust,Content_anger_intensity,Content_anticipation_intensity,Content_disgust_intensity,Content_fear_intensity,Content_joy_intensity,Content_negative_intensity,Content_positive_intensity,Content_sadness_intensity,Content_surprise_intensity,Content_trust_intensity,Content_Valence,Content_Arousal,Content_Dominance,Content_Valence_total_avg,Content_Arousal_total_avg,Content_Dominance_total_avg,Content_intensity_anger_total_avg,Content_intensity_anticipation_total_avg,Content_intensity_disgust_total_avg,Content_intensity_fear_total_avg,Content_intensity_joy_total_avg,Content_intensity_negative_total_avg,Content_intensity_positive_total_avg,Content_intensity_sadness_total_avg,Content_intensity_surprise_total_avg,Content_intensity_trust_total_avg,Content_anger_total_avg,Content_anticipation_total_avg,Content_disgust_total_avg,Content_fear_total_avg,Content_joy_total_avg,Content_negative_total_avg,Content_positive_total_avg,Content_sadness_total_avg,Content_surprise_total_avg,Content_trust_total_avg,Title_Keyword_Count,Content_Keyword_Count,Title_Keyword_Flag,Content_Keyword_Flag,Year
0,[Sports],UCSB End Non-Conf. Portion of Season in Winnin...,2018-01-01,00:37:00,[Spencer Ault],https://dailynexus.com/2018-01-01/80339/,"After breaking through the defense, Jalen Cant...",False,ucsb end non conf portion season winning fashion,breaking defense jalen canty prepares shoot du...,None,None,0.000,0.702,0.298,0.5267,0.048,0.859,0.093,0.9755,Positive,Positive,5,8,0,1,1,0,1,0,1,1,1,1,0.000,0.664,0.0,0.000,0.000,0.0,0.0,0.000,0.0,0.266,0.661600,0.490200,0.555200,0.413500,0.306375,0.347000,0.000000,0.083000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.03325,0.000000,0.125000,0.125,0.000000,0.125000,0.000000,0.125000,0.125000,0.125000,0.125000,178,377,16,12,3,14,5,24,28,8,3,12,7.943,4.593,0.695,0.640,4.944,0.0,0.0,1.022,0.516,5.867,0.593202,0.454537,0.521504,0.380782,0.291772,0.334759,0.021069,0.012183,0.001844,0.001698,0.013114,0.0,0.0,0.002711,0.001369,0.015562,0.042440,0.031830,0.007958,0.037135,0.013263,0.063660,0.074271,0.021220,0.007958,0.031830,0,0,No,No,2018
1,[Sports],Gauchos to Host Long Beach and Cal Poly to Ope...,2018-01-02,00:47:00,[Calvin Thrift],https://dailynexus.com/2018-01-02/gauchos-to-h...,"Sarah Porter dribbles the ball, looking for a ...",False,gaucho host long beach cal poly open big west ...,sarah porter dribble ball looking pas non conf...,None,None,0.000,0.833,0.167,0.3400,0.028,0.859,0.113,0.9952,Posi

## Exploratory Data Analysis

In [51]:
# Explode the 'Section' column to have one row per section per article
final_df_copy = final_df.explode('Section')

# Now, group by 'Year' and 'Section' and count the number of articles
section_df = final_df_copy.groupby(['Year', 'Section'])['URL'].count().reset_index()
section_df.columns = ['Year', 'Section', 'Article_Count']

# Display the resulting dataframe
section_df[section_df['Year'] == 2024]

,Year,Section,Article_Count
63,2024,Artsweek,83
64,2024,Daily Stench,37
65,2024,Data,15
66,2024,Games,16
67,2024,La Vista,18
68,2024,News,206
69,2024,On The Menu,37
70,2024,Opinion,24
71,2024,Photo,6
72,2024,Science & Tech,38


In [99]:
combined_df['Authors'].value_counts()

[Asumi Shuda]                       212
[Evelyn Spence]                     150
[Sindhu Ananthavel]                 135
[Atmika Iyer]                       122
[Max Abrams]                        120
                                   ... 
[Emma Demorest, Vianna Mabanag]       1
[Katherine Swartz, Ashley Rusch]      1
[Video Team, Alexandra Goldberg]      1
[Marie Stevenson]                     1
[Adeline Barron-Merritt]              1
Name: Authors, Length: 1277, dtype: int64

In [101]:
combined_df['Authors'].apply(pd.Series).stack().value_counts()

Asumi Shuda                         273
Sindhu Ananthavel                   198
Holly Rusch                         195
Evelyn Spence                       187
Atmika Iyer                         183
                                   ... 
Austin Beaart                         1
Amon Gus                              1
Emma Barrera                          1
Skylar Moore                          1
Daily Nexus Diversity Task Force      1
Length: 941, dtype: int64

In [97]:
combined_df['Section'].value_counts()

[News]                     2301
[Sports]                    868
[Artsweek]                  675
[Daily Stench]              634
[On The Menu]               599
[Opinion]                   598
[Science & Tech]            431
[La Vista]                  114
[Video]                      73
[Photo]                      71
[Data]                       52
[News, Photo]                33
[News, Opinion]              25
[Data, News]                 23
[Games]                      21
[Artsweek, Photo]            16
[Sports, Video]               4
[Photo, Sports]               3
[News, Science & Tech]        3
[News, Photo, Video]          2
[News, Video]                 2
[Data, Science & Tech]        2
[Photo, Science & Tech]       2
[Artsweek, Video]             2
[Daily Stench, Video]         2
[Opinion, Sports]             1
[Data, Opinion]               1
[Artsweek, News]              1
[Data, News, Opinion]         1
[La Vista, News]              1
[Games, La Vista]             1
[On The 

In [98]:
sections = combined_df['Section'].apply(pd.Series).stack().value_counts()
sections

News              2393
Sports             876
Artsweek           694
Daily Stench       637
Opinion            626
On The Menu        601
Science & Tech     438
Photo              128
La Vista           116
Video               86
Data                79
Games               22
dtype: int64

In [1]:
import pandas as pd
final_df = pd.read_csv('data_storage/full_sentiment_keywords.csv')
final_df['Year'] = final_df['Date'].str[:4].astype(int)

0.03670982482863671

In [8]:
send_1 = pd.read_csv('data_storage/israel_palestine.csv')
# convert date to datetime
send_1['Date'] = pd.to_datetime(send_1['Date'])

# give me df where IP_Title_Keyword_Flaf is Yes and date >= 10/7/2023

send_1 = send_1[(send_1['IP_Title_Keyword_Flag'] == 'Yes') & (send_1['Date'] >= '2023-10-07')]
send_1[['Section', 'Title', 'Authors', 'Date', 'URL', 'Title_Sentiment']].to_csv('chart_storage/Bye Bye/ip.csv', index=False)

In [22]:
send_2 = pd.read_csv('data_storage/full_sentiment_keywords.csv')
send_2['Date'] = pd.to_datetime(send_2['Date']) 
send_2 = send_2[(send_2['Title_Keyword_Flag'] == 'Yes') & (send_2['Date'] >= '2023-06-10')]
# clean section
send_2['Section'] = send_2['Section'].apply(clean_authors)
# Section is a list - see if news is in the list
array = []
for element in send_2['Section']:
    if 'News' in element:
        array.append(True)
    else:
        array.append(False)
send_2['array'] = array
send_2 = send_2[send_2['array'] == True]
send_2[['Section', 'Title', 'Authors', 'Date', 'URL', 'Title_Sentiment']].to_csv('chart_storage/Bye Bye/news.csv', index=False)
send_2[['Section', 'Title', 'Authors', 'Date', 'URL', 'Title_Sentiment']]

,Section,Title,Authors,Date,URL,Title_Sentiment
5791,"[Data, News, Opinion]","Diversity, Equity and Inclusion at the Daily N...","['Siddharth Chattoraj', 'Pricila Flores']",2023-07-12,https://dailynexus.com/2023-07-12/diversity-eq...,Neutral
5889,[News],S.E.A.L. hosts second annual Cultural Identiti...,['Anushka Ghosh Dastidar'],2023-10-12,https://dailynexus.com/2023-10-12/s-e-a-l-host...,Positive
5894,[News],New assembly bill streamlines financial aid fo...,['Asumi Shuda'],2023-10-13,https://dailynexus.com/2023-10-13/new-assembly...,Neutral
5913,[News],Womens Center hires new Associate Director o...,['Anushka Ghosh Dastidar'],2023-10-20,https://dailynexus.com/2023-10-20/womens-cente...,Neutral
5993,[News],UCSB named Fulbright Hispanic-Serving Institut...,['Asumi Shuda'],2023-11-09,https://dailynexus.com/2023-11-09/ucsb-named-f...,Neutral
6089,[News],"A.S., EVPSA hold town hall on UC-wide undocume...","['Lizzy Rager', 'Devin Gowdy']",2024-01-25,https://dailynexus.com/2024-01-25/a-s-evpsa-ho...,Positive
6104,[News],UCSB A.S. C.O.D.E. reveals new Disability Cult...,['Asumi Shuda'],2024-01-26,https://dailynexus.com/2024-01-26/ucsb-a-s-c-o...,Neutral
6151,[News],UC Workers Union calls for UC Regents to dives...,['Asumi Shuda'],2024-02-09,https://dailynexus.com/2024-02-09/uc-workers-u...,Neutral
6158,[News],UCSB HFA hosts Beyond Tokenism: Celebrating ...,['Siduo Zhang'],2024-02-13,https://dailynexus.com/2024-02-13/ucsb-hfa-hos...,Positive
6213,[News],UC-wide union rallies for the UC to break up...,['Lizzy Rager'],2024-02-22,https://dailynexus.com/2024-02-22/uc-wide-unio...,Neutral
